Import Libraries

In [46]:
from pyscf import gto, scf, ao2mo, fci
from GeneralisedWick import *
from scipy.optimize import fsolve

Useful routines for generating coupled cluster expressions

In [47]:
def commutator(operator1, operator2):
    return operator1 * operator2 + (-1) * operator2 * operator1

In [48]:
def BCHSimilarityTransform(H, T, order):
    result = H
    for k in range(order):
        nestedCommutator = H
        for i in range(k + 1):
            nestedCommutator = commutator(nestedCommutator, T)
        result += (1 / factorial(k + 1)) * nestedCommutator
    return result

In [49]:
def projectionManifold(excitationLevel):
    return Tensor('\Phi', ['h'] * excitationLevel, ['p'] * excitationLevel)

Create Tensor objects for the diagram components

In [50]:
fDiagrams = [Tensor('f', ['p'], ['p']), Tensor('f', ['p'], ['h']), Tensor('f', ['h'], ['p']), Tensor('f', ['h'], ['h'])]
vDiagrams = [Tensor('v', ['p', 'p'], ['p', 'p']), Tensor('v', ['h', 'h'], ['h', 'h']), Tensor('v', ['h', 'p'], ['p', 'h']), Tensor('v', ['p', 'h'], ['p', 'p']), Tensor('v', ['h', 'h'], ['h', 'p']), Tensor('v', ['p', 'p'], ['p', 'h']), Tensor('v', ['h', 'p'], ['h', 'h']), Tensor('v', ['p', 'p'], ['h', 'h']), Tensor('v', ['h', 'h'], ['p', 'p'])]
t1 = Tensor('t1', ['p'], ['h'])
t2 = Tensor('t2', ['p', 'p'], ['h', 'h'])

In [51]:
vDiagramsNonAntisym = [Tensor('v', ['p', 'p'], ['p', 'p']), Tensor('v', ['h', 'h'], ['h', 'h']), Tensor('v', ['h', 'p'], ['p', 'h']), Tensor('v', ['h', 'p'], ['h', 'p']), Tensor('v', ['p', 'h'], ['p', 'p']), Tensor('v', ['h', 'h'], ['h', 'p']), Tensor('v', ['p', 'p'], ['p', 'h']), Tensor('v', ['h', 'p'], ['h', 'h']), Tensor('v', ['p', 'p'], ['h', 'h']), Tensor('v', ['h', 'h'], ['p', 'p'])]

In [52]:
projectionSingles = projectionManifold(1)
projectionDoubles = projectionManifold(2)

In [53]:
projectionDoubles.array

AttributeError: 'Tensor' object has no attribute 'array'

In [54]:
projectionSingles.getShape(vacuum)
projectionDoubles.getShape(vacuum)

In [55]:
NonAntisymEnergyTerms = [evaluateWick(BCHSimilarityTransform(vD, (1./4.) * t2, 4), spinFree=True) for vD in vDiagramsNonAntisym]

KeyboardInterrupt: 

In [56]:
for e, ET in enumerate(NonAntisymEnergyTerms):
    print(e, ET)

NameError: name 'NonAntisymEnergyTerms' is not defined

In [ ]:
NonAntisymAmplitudeTerms = [evaluateWick(projectionDoubles * BCHSimilarityTransform(vD, (1./4.) * t2, 4), spinFree=True) for vD in vDiagramsNonAntisym]

In [ ]:
NonAntisymProjectedTerms = NonAntisymAmplitudeTerms
NonAntisymAmplitudeTerms = [copy(term) for term in NonAntisymProjectedTerms]

In [ ]:
NonAntisymProjectedFockTerms = [evaluateWick(projectionDoubles * BCHSimilarityTransform(fD, (1./4.) * t2, 4), spinFree=True) for fD in fDiagrams]

In [ ]:
NonAntisymAmplitudeFockTerms = [copy(term) for term in NonAntisymProjectedFockTerms]
for term in NonAntisymAmplitudeFockTerms:
    for summand in term.summandList:
        summand.vertexList.pop(0)
        summand.tensorList.pop(0)

In [ ]:
for term in NonAntisymAmplitudeTerms:
    for summand in term.summandList:
        summand.vertexList.pop(0)
        summand.tensorList.pop(0)

In [ ]:
print(NonAntisymAmplitudeTerms[8])
contractTensorSum(NonAntisymAmplitudeTerms[8])

4.0 * v_{p_{1}p_{0}}^{h_{1}h_{0}}
 + -2.0 * v_{p_{0}p_{1}}^{h_{1}h_{0}}
 + -2.0 * v_{p_{1}p_{0}}^{h_{0}h_{1}}
 + 4.0 * v_{p_{0}p_{1}}^{h_{0}h_{1}}


array([[[[2.58099771]]]])

In [ ]:
unweightedAmplitudeFockTerms = [contractTensorSum(term) for term in NonAntisymAmplitudeFockTerms if term.summandList != []]
unweightedAmplitudeTerms = [contractTensorSum(term) for term in NonAntisymAmplitudeTerms if term.summandList != []]

In [ ]:
h1

NameError: name 'h1' is not defined

In [ ]:
print(NonAntisymAmplitudeTerms[2])

0.5 * v_{h_{2}p_{1}}^{p_{3}h_{0}}t2_{p_{0}p_{3}}^{h_{2}h_{1}}
 + -1.0 * v_{h_{2}p_{1}}^{p_{3}h_{0}}t2_{p_{3}p_{0}}^{h_{2}h_{1}}
 + -1.0 * v_{h_{2}p_{0}}^{p_{3}h_{0}}t2_{p_{1}p_{3}}^{h_{2}h_{1}}
 + 2.0 * v_{h_{2}p_{0}}^{p_{3}h_{0}}t2_{p_{3}p_{1}}^{h_{2}h_{1}}
 + -1.0 * v_{h_{2}p_{1}}^{p_{3}h_{0}}t2_{p_{0}p_{3}}^{h_{1}h_{2}}
 + 0.5 * v_{h_{2}p_{1}}^{p_{3}h_{0}}t2_{p_{3}p_{0}}^{h_{1}h_{2}}
 + 2.0 * v_{h_{2}p_{0}}^{p_{3}h_{0}}t2_{p_{1}p_{3}}^{h_{1}h_{2}}
 + -1.0 * v_{h_{2}p_{0}}^{p_{3}h_{0}}t2_{p_{3}p_{1}}^{h_{1}h_{2}}
 + -1.0 * v_{h_{2}p_{1}}^{p_{3}h_{1}}t2_{p_{0}p_{3}}^{h_{2}h_{0}}
 + 2.0 * v_{h_{2}p_{1}}^{p_{3}h_{1}}t2_{p_{3}p_{0}}^{h_{2}h_{0}}
 + 0.5 * v_{h_{2}p_{0}}^{p_{3}h_{1}}t2_{p_{1}p_{3}}^{h_{2}h_{0}}
 + -1.0 * v_{h_{2}p_{0}}^{p_{3}h_{1}}t2_{p_{3}p_{1}}^{h_{2}h_{0}}
 + 2.0 * v_{h_{2}p_{1}}^{p_{3}h_{1}}t2_{p_{0}p_{3}}^{h_{0}h_{2}}
 + -1.0 * v_{h_{2}p_{1}}^{p_{3}h_{1}}t2_{p_{3}p_{0}}^{h_{0}h_{2}}
 + -1.0 * v_{h_{2}p_{0}}^{p_{3}h_{1}}t2_{p_{1}p_{3}}^{h_{0}h_{2}}
 + 0.5 * v_{h_{2}p_{

In [ ]:
for s, summand in enumerate(NonAntisymAmplitudeTerms[2].summandList):
    if s == 3 or s == 6 or s == 9 or s == 12:
        summand.prefactor = 2.0

In [ ]:
contractTensorSum(NonAntisymAmplitudeTerms[2])

array([[[[-0.17765866]]]])

In [ ]:
print(*unweightedAmplitudeFockTerms)
print(*unweightedAmplitudeTerms)

[[[[0.14695891]]]] [[[[-0.32760552]]]]
[[[[-0.18671348]]]] [[[[-0.18007061]]]] [[[[-0.05163105]]]] [[[[0.1032621]]]] [[[[2.58099771]]]] [[[[-0.01222884]]]]


In [ ]:
unweightedAmplitudeFockTerms[0] + unweightedAmplitudeFockTerms[1] + (1./4.) * unweightedAmplitudeTerms[0] + (1./4.) * unweightedAmplitudeTerms[1] + (1./1.) * unweightedAmplitudeTerms[2] + (1./1.) * unweightedAmplitudeTerms[3] + (1./4.) * unweightedAmplitudeTerms[4] + (1./4.) * unweightedAmplitudeTerms[5]

array([[[[0.42148064]]]])

In [ ]:
(1./4.) * unweightedAmplitudeTerms[4]

array([[[[0.64524943]]]])

In [ ]:
(1./4.) * unweightedAmplitudeTerms[5] / trueAmplitude

array([[[[0.02220733]]]])

Construct Normal-ordered Hamiltonian

In [57]:
NormalOrderedFock = sum(fDiagrams)

In [58]:
NormalOrdered2BodyH = (1./4.) * vDiagrams[0] + (1./4.) * vDiagrams[1] + vDiagrams[2] + (1./2.) * vDiagrams[3] + (1./2.) * vDiagrams[4] + (1./2.) * vDiagrams[5] + (1./2.) * vDiagrams[6] + (1./4.) * vDiagrams[7] + (1./4.) * vDiagrams[8]

In [59]:
NormalOrdered2BodyHNonAntisym = (1./4.) * vDiagramsNonAntisym[0] + (1./4.) * vDiagramsNonAntisym[1] + (1./2.) * vDiagramsNonAntisym[2] + (1./2.) * vDiagramsNonAntisym[3] + (1./2.) * vDiagramsNonAntisym[4] + (1./2.) * vDiagramsNonAntisym[5] + (1./2.) * vDiagramsNonAntisym[6] + (1./2.) * vDiagramsNonAntisym[7] + (1./4.) * vDiagramsNonAntisym[8] + (1./4.) * vDiagramsNonAntisym[9]

In [60]:
NormalOrderedHamiltonian = NormalOrderedFock + NormalOrdered2BodyH

In [61]:
NormalOrderedHamiltonianNonAntisym = NormalOrderedFock + NormalOrdered2BodyHNonAntisym

Generate coupled cluster equations from similarity-transformed normal-ordered Hamiltonian

In [62]:
CCDtransformH = sum([BCHSimilarityTransform(d, (1./4.) * t2, 4) for d in fDiagrams] + [BCHSimilarityTransform((1./2.) * d, (1./4.) * t2, 4) for d in vDiagrams])
weightFactorCCDtransformH = BCHSimilarityTransform(NormalOrderedHamiltonian, (1./4.) * t2, 4)

In [63]:
CCDtransformHNonAntisym = BCHSimilarityTransform(NormalOrderedHamiltonianNonAntisym, (1./4.) * t2, 4)
CCDEnergyEquationNonAntisym = evaluateWick(CCDtransformHNonAntisym, spinFree=True)

KeyboardInterrupt: 

In [64]:
projectedCCDtransformH = projectionDoubles * CCDtransformH
weightFactorProjectedCCDtransformH = projectionDoubles * weightFactorCCDtransformH

In [65]:
CCDEnergyEquation = evaluateWick(CCDtransformH, spinFree=True)
weightFactorCCDEnergyEquation = evaluateWick(weightFactorCCDtransformH, spinFree=True)

KeyboardInterrupt: 

In [ ]:
print(weightFactorCCDEnergyEquation)

0.25 * v_{h_{0}h_{1}}^{p_{0}p_{1}}t2_{p_{1}p_{0}}^{h_{1}h_{0}}
 + -0.125 * v_{h_{0}h_{1}}^{p_{0}p_{1}}t2_{p_{0}p_{1}}^{h_{1}h_{0}}
 + -0.125 * v_{h_{0}h_{1}}^{p_{0}p_{1}}t2_{p_{1}p_{0}}^{h_{0}h_{1}}
 + 0.25 * v_{h_{0}h_{1}}^{p_{0}p_{1}}t2_{p_{0}p_{1}}^{h_{0}h_{1}}


In [ ]:
CCDAmplitudeEquation = evaluateWick(projectedCCDtransformH, spinFree=True)

In [ ]:
weightFactorCCDAmplitudeEquation = evaluateWick(weightFactorProjectedCCDtransformH, spinFree=True)

In [ ]:
projectedCCDtransformHNonAntisym = projectionDoubles * CCDtransformHNonAntisym
CCDAmplitudeEquationNonAntisym = evaluateWick(projectedCCDtransformHNonAntisym, spinFree=True)

In [ ]:
print(testWeightFactorCCDAmplitudeContractions)

1.0 * v_{p_{1}p_{0}}^{h_{1}h_{0}}
 + -0.5 * v_{p_{0}p_{1}}^{h_{1}h_{0}}
 + -0.5 * v_{p_{1}p_{0}}^{h_{0}h_{1}}
 + 1.0 * v_{p_{0}p_{1}}^{h_{0}h_{1}}
 + -0.5 * f_{p_{1}}^{p_{3}}t2_{p_{0}p_{3}}^{h_{1}h_{0}}
 + 1.0 * f_{p_{1}}^{p_{3}}t2_{p_{3}p_{0}}^{h_{1}h_{0}}
 + 1.0 * f_{p_{0}}^{p_{3}}t2_{p_{1}p_{3}}^{h_{1}h_{0}}
 + -0.5 * f_{p_{0}}^{p_{3}}t2_{p_{3}p_{1}}^{h_{1}h_{0}}
 + 1.0 * f_{p_{1}}^{p_{3}}t2_{p_{0}p_{3}}^{h_{0}h_{1}}
 + -0.5 * f_{p_{1}}^{p_{3}}t2_{p_{3}p_{0}}^{h_{0}h_{1}}
 + -0.5 * f_{p_{0}}^{p_{3}}t2_{p_{1}p_{3}}^{h_{0}h_{1}}
 + 1.0 * f_{p_{0}}^{p_{3}}t2_{p_{3}p_{1}}^{h_{0}h_{1}}
 + 0.5 * f_{h_{2}}^{h_{0}}t2_{p_{1}p_{0}}^{h_{2}h_{1}}
 + -1.0 * f_{h_{2}}^{h_{0}}t2_{p_{0}p_{1}}^{h_{2}h_{1}}
 + -1.0 * f_{h_{2}}^{h_{0}}t2_{p_{1}p_{0}}^{h_{1}h_{2}}
 + 0.5 * f_{h_{2}}^{h_{0}}t2_{p_{0}p_{1}}^{h_{1}h_{2}}
 + -1.0 * f_{h_{2}}^{h_{1}}t2_{p_{1}p_{0}}^{h_{2}h_{0}}
 + 0.5 * f_{h_{2}}^{h_{1}}t2_{p_{0}p_{1}}^{h_{2}h_{0}}
 + 0.5 * f_{h_{2}}^{h_{1}}t2_{p_{1}p_{0}}^{h_{0}h_{2}}
 + -1.0 * f_{h_{2}}^{

In [ ]:
CCStransformH = sum([BCHSimilarityTransform(d, t1, 4) for d in fDiagrams] + [BCHSimilarityTransform((1./2.) * d, t1, 4) for d in vDiagrams])

In [ ]:
projectedCCStransformH = projectionSingles * CCStransformH

In [ ]:
CCSEnergyEquation = evaluateWick(CCStransformH, spinFree=True)

In [ ]:
print(CCSEnergyEquation)

2.0 * f_{h_{0}}^{p_{0}}t1_{p_{0}}^{h_{0}}
 + -0.5 * v_{h_{0}h_{1}}^{p_{0}p_{1}}t1_{p_{1}}^{h_{0}}t1_{p_{0}}^{h_{1}}
 + 1.0 * v_{h_{0}h_{1}}^{p_{0}p_{1}}t1_{p_{0}}^{h_{0}}t1_{p_{1}}^{h_{1}}
 + 1.0 * v_{h_{0}h_{1}}^{p_{0}p_{1}}t1_{p_{1}}^{h_{1}}t1_{p_{0}}^{h_{0}}
 + -0.5 * v_{h_{0}h_{1}}^{p_{0}p_{1}}t1_{p_{0}}^{h_{1}}t1_{p_{1}}^{h_{0}}


In [ ]:
CCSAmplitudeEquation = evaluateWick(projectedCCStransformH, spinFree=True)

In [ ]:
print(CCSAmplitudeEquation)

2.0 * \Phi_{h_{0}}^{p_{0}}f_{p_{0}}^{p_{2}}t1_{p_{2}}^{h_{0}}
 + 2.0 * \Phi_{h_{0}}^{p_{0}}f_{p_{0}}^{h_{0}}
 + 2.0 * \Phi_{h_{0}}^{p_{0}}f_{h_{1}}^{p_{1}}t1_{p_{0}}^{h_{0}}t1_{p_{1}}^{h_{1}}
 + -1.0 * \Phi_{h_{0}}^{p_{0}}f_{h_{1}}^{p_{1}}t1_{p_{1}}^{h_{0}}t1_{p_{0}}^{h_{1}}
 + -1.0 * \Phi_{h_{0}}^{p_{0}}f_{h_{1}}^{p_{1}}t1_{p_{0}}^{h_{1}}t1_{p_{1}}^{h_{0}}
 + 2.0 * \Phi_{h_{0}}^{p_{0}}f_{h_{1}}^{p_{1}}t1_{p_{1}}^{h_{1}}t1_{p_{0}}^{h_{0}}
 + -2.0 * \Phi_{h_{0}}^{p_{0}}t1_{p_{0}}^{h_{0}}f_{h_{2}}^{p_{2}}t1_{p_{2}}^{h_{2}}
 + -2.0 * \Phi_{h_{0}}^{p_{0}}t1_{p_{0}}^{h_{0}}f_{h_{2}}^{p_{2}}t1_{p_{2}}^{h_{2}}
 + -2.0 * \Phi_{h_{0}}^{p_{0}}f_{h_{1}}^{h_{0}}t1_{p_{0}}^{h_{1}}
 + 2.0 * \Phi_{h_{0}}^{p_{0}}v_{h_{1}p_{0}}^{p_{2}h_{0}}t1_{p_{2}}^{h_{1}}
 + -0.5 * \Phi_{h_{0}}^{p_{0}}v_{p_{0}h_{1}}^{p_{2}p_{3}}t1_{p_{3}}^{h_{0}}t1_{p_{2}}^{h_{1}}
 + 1.0 * \Phi_{h_{0}}^{p_{0}}v_{p_{0}h_{1}}^{p_{2}p_{3}}t1_{p_{2}}^{h_{0}}t1_{p_{3}}^{h_{1}}
 + 1.0 * \Phi_{h_{0}}^{p_{0}}v_{p_{0}h_{1}}^{p_{2}p_{3}}t1_{p

Set up H2 molecule in minimal basis, calculate SCF reference

In [66]:
bohr = 0.529177249

H2sep = 1.605 * bohr

mol = gto.Mole()
mol.verbose = 1
mol.atom = 'H 0 0 0; H 0 0 ' + str(H2sep)
mol.basis = 'sto-3g'
mol.spin = 0
mol.build()

Enuc = mol.energy_nuc()

mf = scf.ROHF(mol)
mf.kernel()

h1 = mf.mo_coeff.T.dot(mf.get_hcore()).dot(mf.mo_coeff)
eri = ao2mo.kernel(mol, mf.mo_coeff, compact=False)

cisolver = fci.FCI(mol, mf.mo_coeff)

Norbs = mol.nao
Nocc = mf.nelectron_alpha
vacuum = [1] * Nocc + [0] * (Norbs - Nocc)

h2 = eri.reshape((Norbs, Norbs, Norbs, Norbs))

In [67]:
vDiagramsNonAntisym[8].array * (-1. / (mf.mo_energy[1] + mf.mo_energy[1] - mf.mo_energy[0] - mf.mo_energy[0]))

AttributeError: 'Tensor' object has no attribute 'array'

In [68]:
from pyscf import cc
ccdsolver = cc.CCSD(mf)

In [69]:
CCDresults = ccdsolver.kernel()

In [70]:
print(CCDresults[2][1][0,0,0,0])

-0.13766665750371915


In [71]:
trueAmplitude = CCDresults[2][1][0,0,0,0]

In [72]:
ao2mo.kernel(mf.mol, mf.mo_coeff)

array([[6.54009511e-01, 8.32667268e-17, 6.45249427e-01],
       [1.04083409e-16, 1.87521981e-01, 5.55111512e-17],
       [6.45249427e-01, 1.38777878e-16, 6.78136184e-01]])

In [73]:
def get2bodyHamiltonian(mf_):
    eri = ao2mo.kernel(mf_.mol, mf_.mo_coeff)
    Norbs_ = mf_.mol.nao
    hamiltonian2Body = np.zeros((Norbs_, Norbs_, Norbs_, Norbs_))
    for p in range(Norbs_):
        for q in range(Norbs_):
            for r in range(p + 1):
                for s in range(q + 1):
                    x = int(p + Norbs_ * q - 0.5 * q * (q + 1))
                    y = int(r + Norbs_ * s - 0.5 * s * (s + 1))
                    if p == r and q == s:
                        print(p,q,r,s,eri[x,y])
                        hamiltonian2Body[p,q,r,s] = eri[x, y]
                    elif p == r:
                        print(p,q,r,s,eri[x,y])
                        print(p,s,r,q,np.conjugate(eri[x,y]))
                        hamiltonian2Body[p,q,r,s] = eri[x, y]
                        hamiltonian2Body[p,s,r,q] = np.conjugate(eri[x, y])
                    elif q == s:
                        print(p,q,r,s,eri[x,y])
                        print(r,q,p,s,np.conjugate(eri[x,y]))
                        hamiltonian2Body[p,q,r,s] = eri[x, y]
                        hamiltonian2Body[r,q,p,s] = np.conjugate(eri[x, y])
                    else:
                        print(p,q,r,s,eri[x,y])
                        print(r,q,p,s,np.conjugate(eri[x,y]))
                        print(p,s,r,q,np.conjugate(eri[x,y]))
                        print(r,s,p,q,eri[x,y])
                        hamiltonian2Body[p,q,r,s] = eri[x, y]
                        hamiltonian2Body[r,q,p,s] = np.conjugate(eri[x, y])
                        hamiltonian2Body[p,s,r,q] = np.conjugate(eri[x, y])
                        hamiltonian2Body[r,s,p,q] = eri[x, y]
                    print(x,y)
    return hamiltonian2Body

In [74]:
def newGet2bodyHamiltonian(mf_):
    eri = ao2mo.kernel(mf_.mol, mf_.mo_coeff)
    Norbs_ = mf_.mol.nao
    hamiltonian2Body = np.zeros((Norbs_, Norbs_, Norbs_, Norbs_))
    for s in range(Norbs_):
        for q in range(Norbs_):
            for r in range(s + 1):
                for p in range(q + 1):
                    x = int(p + Norbs_ * q - 0.5 * q * (q + 1))
                    y = int(r + Norbs_ * s - 0.5 * s * (s + 1))
                    if p == q and r == s:
                        print(p,q,r,s,eri[x,y])
                        hamiltonian2Body[p,q,r,s] = eri[x, y]
                    elif p == q or r == s:
                        print(p,q,r,s,eri[x,y])
                        print(q,p,s,r,eri[x,y])
#                        print(p,s,r,q,np.conjugate(eri[x,y]))
                        hamiltonian2Body[p,q,r,s] = eri[x, y]
                        hamiltonian2Body[q,p,s,r] = eri[x, y]
#                        hamiltonian2Body[p,s,r,q] = np.conjugate(eri[x, y])
#                    elif q == s:
#                        print(p,q,r,s,eri[x,y])
#                        print(r,q,p,s,np.conjugate(eri[x,y]))
#                        hamiltonian2Body[p,q,r,s] = eri[x, y]
#                        hamiltonian2Body[r,q,p,s] = np.conjugate(eri[x, y])
                    else:
                        print(p,q,r,s,eri[x,y])
                        print(q,p,s,r,eri[x,y])
                        print(p,q,s,r,eri[x,y])
                        print(q,p,r,s,eri[x,y])
#                        print(r,q,p,s,np.conjugate(eri[x,y]))
#                        print(p,s,r,q,np.conjugate(eri[x,y]))
#                        print(r,s,p,q,eri[x,y])
                        hamiltonian2Body[p,q,r,s] = eri[x, y]
                        hamiltonian2Body[q,p,s,r] = eri[x, y]
                        hamiltonian2Body[p,q,s,r] = eri[x, y]
                        hamiltonian2Body[q,p,r,s] = eri[x, y]
#                        hamiltonian2Body[r,q,p,s] = np.conjugate(eri[x, y])
#                        hamiltonian2Body[p,s,r,q] = np.conjugate(eri[x, y])
#                        hamiltonian2Body[r,s,p,q] = eri[x, y]
                    print(x,y)
    return hamiltonian2Body

In [75]:
eri_ao = mol.intor('int2e_sph')
h2 = ao2mo.incore.full(eri_ao, mf.mo_coeff)


In [76]:
newGet2bodyHamiltonian(mf)

0 0 0 0 0.6540095113561238
0 0
0 1 0 0 1.0408340855860843e-16
1 0 0 0 1.0408340855860843e-16
1 0
1 1 0 0 0.6452494265884844
2 0
0 0 0 1 8.326672684688674e-17
0 0 1 0 8.326672684688674e-17
0 1
0 0 1 1 0.6452494265884848
0 2
0 1 0 1 0.18752198090270858
1 0 1 0 0.18752198090270858
0 1 1 0 0.18752198090270858
1 0 0 1 0.18752198090270858
1 1
1 1 0 1 1.3877787807814457e-16
1 1 1 0 1.3877787807814457e-16
2 1
0 1 1 1 5.551115123125783e-17
1 0 1 1 5.551115123125783e-17
1 2
1 1 1 1 0.6781361836853057
2 2


array([[[[6.54009511e-01, 8.32667268e-17],
         [8.32667268e-17, 6.45249427e-01]],

        [[1.04083409e-16, 1.87521981e-01],
         [1.87521981e-01, 5.55111512e-17]]],


       [[[1.04083409e-16, 1.87521981e-01],
         [1.87521981e-01, 5.55111512e-17]],

        [[6.45249427e-01, 1.38777878e-16],
         [1.38777878e-16, 6.78136184e-01]]]])

In [77]:
h2 = get2bodyHamiltonian(mf)
print(h2)

0 0 0 0 0.6540095113561238
0 0
0 1 0 0 1.0408340855860843e-16
0 0 0 1 1.0408340855860843e-16
1 0
0 1 0 1 0.18752198090270858
1 1
1 0 0 0 1.0408340855860843e-16
0 0 1 0 1.0408340855860843e-16
1 0
1 0 1 0 0.18752198090270858
1 1
1 1 0 0 0.6452494265884844
0 1 1 0 0.6452494265884844
1 0 0 1 0.6452494265884844
0 0 1 1 0.6452494265884844
2 0
1 1 0 1 1.3877787807814457e-16
0 1 1 1 1.3877787807814457e-16
2 1
1 1 1 0 1.3877787807814457e-16
1 0 1 1 1.3877787807814457e-16
2 1
1 1 1 1 0.6781361836853057
2 2
[[[[6.54009511e-01 1.04083409e-16]
   [1.04083409e-16 6.45249427e-01]]

  [[1.04083409e-16 1.87521981e-01]
   [6.45249427e-01 1.38777878e-16]]]


 [[[1.04083409e-16 6.45249427e-01]
   [1.87521981e-01 1.38777878e-16]]

  [[6.45249427e-01 1.38777878e-16]
   [1.38777878e-16 6.78136184e-01]]]]


In [78]:
ao2mo.kernel(mf.mol, mf.mo_coeff)

array([[6.54009511e-01, 8.32667268e-17, 6.45249427e-01],
       [1.04083409e-16, 1.87521981e-01, 5.55111512e-17],
       [6.45249427e-01, 1.38777878e-16, 6.78136184e-01]])

In [79]:
eri

array([[6.54009511e-01, 1.11022302e-16, 8.32667268e-17, 6.45249427e-01],
       [1.17961196e-16, 1.87521981e-01, 1.87521981e-01, 1.11022302e-16],
       [1.04083409e-16, 1.87521981e-01, 1.87521981e-01, 5.55111512e-17],
       [6.45249427e-01, 2.22044605e-16, 1.38777878e-16, 6.78136184e-01]])

In [80]:
h2 = eri.reshape((Norbs, Norbs, Norbs, Norbs), order='C')
print(h2)

[[[[6.54009511e-01 1.11022302e-16]
   [8.32667268e-17 6.45249427e-01]]

  [[1.17961196e-16 1.87521981e-01]
   [1.87521981e-01 1.11022302e-16]]]


 [[[1.04083409e-16 1.87521981e-01]
   [1.87521981e-01 5.55111512e-17]]

  [[6.45249427e-01 2.22044605e-16]
   [1.38777878e-16 6.78136184e-01]]]]


In [81]:
h2

array([[[[6.54009511e-01, 1.11022302e-16],
         [8.32667268e-17, 6.45249427e-01]],

        [[1.17961196e-16, 1.87521981e-01],
         [1.87521981e-01, 1.11022302e-16]]],


       [[[1.04083409e-16, 1.87521981e-01],
         [1.87521981e-01, 5.55111512e-17]],

        [[6.45249427e-01, 2.22044605e-16],
         [1.38777878e-16, 6.78136184e-01]]]])

In [82]:
vDiagramsNonAntisym[0].getShape(vacuum)
vDiagramsNonAntisym[0].array = h2[Nocc:, Nocc:, Nocc:, Nocc:]

vDiagramsNonAntisym[1].getShape(vacuum)
vDiagramsNonAntisym[1].array = h2[:Nocc, :Nocc, :Nocc, :Nocc]

vDiagramsNonAntisym[2].getShape(vacuum)
vDiagramsNonAntisym[2].array = h2[:Nocc, Nocc:, Nocc:, :Nocc]

vDiagramsNonAntisym[3].getShape(vacuum)
vDiagramsNonAntisym[3].array = h2[:Nocc, Nocc:, :Nocc, Nocc:]

vDiagramsNonAntisym[4].getShape(vacuum)
vDiagramsNonAntisym[4].array = h2[Nocc:, :Nocc, Nocc:, Nocc:]

vDiagramsNonAntisym[5].getShape(vacuum)
vDiagramsNonAntisym[5].array = h2[:Nocc, :Nocc, :Nocc, Nocc:]

vDiagramsNonAntisym[6].getShape(vacuum)
vDiagramsNonAntisym[6].array = h2[Nocc:, Nocc:, Nocc:, :Nocc]

vDiagramsNonAntisym[7].getShape(vacuum)
vDiagramsNonAntisym[7].array = h2[:Nocc, Nocc:, :Nocc, :Nocc]

vDiagramsNonAntisym[8].getShape(vacuum)
vDiagramsNonAntisym[8].array = h2[Nocc:, Nocc:, :Nocc, :Nocc]

vDiagramsNonAntisym[9].getShape(vacuum)
vDiagramsNonAntisym[9].array = h2[:Nocc, :Nocc, Nocc:, Nocc:]

t1.getShape(vacuum)

t2.getShape(vacuum)

In [83]:
fDiagrams[0].getShape(vacuum)
fDiagrams[0].array = h1[Nocc:, Nocc:]

fDiagrams[1].getShape(vacuum)
fDiagrams[1].array = h1[Nocc:, :Nocc]

fDiagrams[2].getShape(vacuum)
fDiagrams[2].array = h1[:Nocc, Nocc:]

fDiagrams[3].getShape(vacuum)
fDiagrams[3].array = h1[:Nocc, :Nocc]

vDiagrams[0].getShape(vacuum)
vDiagrams[0].array = h2[Nocc:, Nocc:, Nocc:, Nocc:]

vDiagrams[1].getShape(vacuum)
vDiagrams[1].array = h2[:Nocc, :Nocc, :Nocc, :Nocc]

vDiagrams[2].getShape(vacuum)
vDiagrams[2].array = h2[:Nocc, Nocc:, Nocc:, :Nocc]

vDiagrams[3].getShape(vacuum)
vDiagrams[3].array = h2[Nocc:, :Nocc, Nocc:, Nocc:]

vDiagrams[4].getShape(vacuum)
vDiagrams[4].array = h2[:Nocc, :Nocc, :Nocc, Nocc:]

vDiagrams[5].getShape(vacuum)
vDiagrams[5].array = h2[Nocc:, Nocc:, Nocc:, :Nocc]

vDiagrams[6].getShape(vacuum)
vDiagrams[6].array = h2[:Nocc, Nocc:, :Nocc, :Nocc]

vDiagrams[7].getShape(vacuum)
vDiagrams[7].array = h2[Nocc:, Nocc:, :Nocc, :Nocc]

vDiagrams[8].getShape(vacuum)
vDiagrams[8].array = h2[:Nocc, :Nocc, Nocc:, Nocc:]

t1.getShape(vacuum)

t2.getShape(vacuum)

In [84]:
for d in vDiagrams:
    print(d.array, d.lowerIndexTypes, d.upperIndexTypes)

[[[[0.67813618]]]] ['p', 'p'] ['p', 'p']
[[[[0.65400951]]]] ['h', 'h'] ['h', 'h']
[[[[0.18752198]]]] ['h', 'p'] ['p', 'h']
[[[[5.55111512e-17]]]] ['p', 'h'] ['p', 'p']
[[[[1.11022302e-16]]]] ['h', 'h'] ['h', 'p']
[[[[1.38777878e-16]]]] ['p', 'p'] ['p', 'h']
[[[[1.17961196e-16]]]] ['h', 'p'] ['h', 'h']
[[[[0.64524943]]]] ['p', 'p'] ['h', 'h']
[[[[0.64524943]]]] ['h', 'h'] ['p', 'p']


In [85]:
fDiagrams[3].array

array([[-1.18985062]])

In [86]:
print(CCDEnergyEquation)

NameError: name 'CCDEnergyEquation' is not defined

In [87]:
t2.array[0,0,0,0] = trueAmplitude

In [ ]:
#t2.array[0,0,0,0] = -0.06883332874447325
#CCDElectronicEnergy = sum([getContractedArray(term)[0] for term in CCDEnergyEquation.summandList])
#CCDElectronicEnergy = contractTensorSum(CCDEnergyEquation)
#CCDElectronicEnergy = contractTensorSum(weightFactorCCDEnergyEquation)
CCDElectronicEnergy = contractTensorSum(CCDEnergyEquationNonAntisym)
print(CCDElectronicEnergy)
print(mf.energy_tot() + CCDElectronicEnergy)

-0.022207332953657028
-1.1248461441453808


In [ ]:
print(CCDAmplitudeEquation)

-0.5 * \Phi_{h_{0}h_{1}}^{p_{0}p_{1}}f_{p_{1}}^{p_{3}}t2_{p_{0}p_{3}}^{h_{1}h_{0}}
 + 1.0 * \Phi_{h_{0}h_{1}}^{p_{0}p_{1}}f_{p_{1}}^{p_{3}}t2_{p_{3}p_{0}}^{h_{1}h_{0}}
 + 1.0 * \Phi_{h_{0}h_{1}}^{p_{0}p_{1}}f_{p_{0}}^{p_{3}}t2_{p_{1}p_{3}}^{h_{1}h_{0}}
 + -0.5 * \Phi_{h_{0}h_{1}}^{p_{0}p_{1}}f_{p_{0}}^{p_{3}}t2_{p_{3}p_{1}}^{h_{1}h_{0}}
 + 1.0 * \Phi_{h_{0}h_{1}}^{p_{0}p_{1}}f_{p_{1}}^{p_{3}}t2_{p_{0}p_{3}}^{h_{0}h_{1}}
 + -0.5 * \Phi_{h_{0}h_{1}}^{p_{0}p_{1}}f_{p_{1}}^{p_{3}}t2_{p_{3}p_{0}}^{h_{0}h_{1}}
 + -0.5 * \Phi_{h_{0}h_{1}}^{p_{0}p_{1}}f_{p_{0}}^{p_{3}}t2_{p_{1}p_{3}}^{h_{0}h_{1}}
 + 1.0 * \Phi_{h_{0}h_{1}}^{p_{0}p_{1}}f_{p_{0}}^{p_{3}}t2_{p_{3}p_{1}}^{h_{0}h_{1}}
 + 0.5 * \Phi_{h_{0}h_{1}}^{p_{0}p_{1}}f_{h_{2}}^{h_{0}}t2_{p_{1}p_{0}}^{h_{2}h_{1}}
 + -1.0 * \Phi_{h_{0}h_{1}}^{p_{0}p_{1}}f_{h_{2}}^{h_{0}}t2_{p_{0}p_{1}}^{h_{2}h_{1}}
 + -1.0 * \Phi_{h_{0}h_{1}}^{p_{0}p_{1}}f_{h_{2}}^{h_{0}}t2_{p_{1}p_{0}}^{h_{1}h_{2}}
 + 0.5 * \Phi_{h_{0}h_{1}}^{p_{0}p_{1}}f_{h_{2}}^{h_{0}}t2_{p_

In [ ]:
len(CCDAmplitudeEquation.summandList)

660

In [ ]:
testCCDAmplitudeContractions = copy(CCDAmplitudeEquation)

In [ ]:
testWeightFactorCCDAmplitudeContractions = copy(weightFactorCCDAmplitudeEquation)

In [ ]:
testCCSAmplitudeContractions = copy(CCSAmplitudeEquation)

In [ ]:
for summand in testCCDAmplitudeContractions.summandList:
    summand.vertexList.pop(0)
    summand.tensorList.pop(0)

In [ ]:
for summand in testWeightFactorCCDAmplitudeContractions.summandList:
    summand.vertexList.pop(0)
    summand.tensorList.pop(0)

In [ ]:
for summand in testCCSAmplitudeContractions.summandList:
    summand.vertexList.pop(0)
    summand.tensorList.pop(0)

In [ ]:
CCSResidual = contractTensorSum(testCCSAmplitudeContractions)
print(CCSResidual)

[[-9.42847138e-17]]


In [ ]:
print(testCCDAmplitudeContractions)

-0.5 * f_{p_{1}}^{p_{3}}t2_{p_{0}p_{3}}^{h_{1}h_{0}}
 + 1.0 * f_{p_{1}}^{p_{3}}t2_{p_{3}p_{0}}^{h_{1}h_{0}}
 + 1.0 * f_{p_{0}}^{p_{3}}t2_{p_{1}p_{3}}^{h_{1}h_{0}}
 + -0.5 * f_{p_{0}}^{p_{3}}t2_{p_{3}p_{1}}^{h_{1}h_{0}}
 + 1.0 * f_{p_{1}}^{p_{3}}t2_{p_{0}p_{3}}^{h_{0}h_{1}}
 + -0.5 * f_{p_{1}}^{p_{3}}t2_{p_{3}p_{0}}^{h_{0}h_{1}}
 + -0.5 * f_{p_{0}}^{p_{3}}t2_{p_{1}p_{3}}^{h_{0}h_{1}}
 + 1.0 * f_{p_{0}}^{p_{3}}t2_{p_{3}p_{1}}^{h_{0}h_{1}}
 + 0.5 * f_{h_{2}}^{h_{0}}t2_{p_{1}p_{0}}^{h_{2}h_{1}}
 + -1.0 * f_{h_{2}}^{h_{0}}t2_{p_{0}p_{1}}^{h_{2}h_{1}}
 + -1.0 * f_{h_{2}}^{h_{0}}t2_{p_{1}p_{0}}^{h_{1}h_{2}}
 + 0.5 * f_{h_{2}}^{h_{0}}t2_{p_{0}p_{1}}^{h_{1}h_{2}}
 + -1.0 * f_{h_{2}}^{h_{1}}t2_{p_{1}p_{0}}^{h_{2}h_{0}}
 + 0.5 * f_{h_{2}}^{h_{1}}t2_{p_{0}p_{1}}^{h_{2}h_{0}}
 + 0.5 * f_{h_{2}}^{h_{1}}t2_{p_{1}p_{0}}^{h_{0}h_{2}}
 + -1.0 * f_{h_{2}}^{h_{1}}t2_{p_{0}p_{1}}^{h_{0}h_{2}}
 + -0.25 * v_{p_{1}p_{0}}^{p_{4}p_{5}}t2_{p_{5}p_{4}}^{h_{1}h_{0}}
 + 0.5 * v_{p_{1}p_{0}}^{p_{4}p_{5}}t2_{p_{4}p_{

In [ ]:
testCCDAmplitudeContractionsNonAntiSym = copy(CCDAmplitudeEquationNonAntisym)
for summand in testCCDAmplitudeContractionsNonAntiSym.summandList:
    summand.vertexList.pop(0)
    summand.tensorList.pop(0)

In [ ]:
contractTensorSum(testCCDAmplitudeContractionsNonAntiSym)

array([[[[-3.92047506e-16]]]])

In [ ]:
len(testCCDAmplitudeContractionsNonAntiSym.summandList)

676

In [ ]:
testCCDAmplitudeContractions.summandList[0].tensorList

In [88]:
print(t2.array)

[[[[-0.13766666]]]]


In [89]:
t2.array[0,0,0,0] = -0.06883332874447325

In [90]:
-0.06883332874447325 * 4

-0.275333314977893

In [91]:
#CCDResidual = sum([getContractedArray(term) for term in CCDAmplitudeEquation.summandList])
#CCDResidual = contractTensorSum(testCCDAmplitudeContractions)
CCDResidual = contractTensorSum(testWeightFactorCCDAmplitudeContractions)

NameError: name 'testWeightFactorCCDAmplitudeContractions' is not defined

In [ ]:
print(CCDResidual)

[[[[0.31821854]]]]


In [ ]:
def testCCDamplitude(t):
    t2.array[0,0,0,0] = t
#    CCDResidual = sum([getContractedArray(term) for term in CCDAmplitudeEquation.summandList])
#    CCDResidual = contractTensorSum(testCCDAmplitudeContractions)
    CCDResidual = contractTensorSum(testCCDAmplitudeContractionsNonAntiSym)
#    CCDResidual = contractTensorSum(testWeightFactorCCDAmplitudeContractions)
#    CCDResidual = contractTensorSum(newAmplitudeTensor) + eri[0,3]
    return CCDResidual[0,0,0,0]

In [ ]:
amplitude = fsolve(testCCDamplitude, 0)

In [ ]:
amplitude

array([-0.28129445])

In [ ]:
t2.array

array([[[[-0.13766666]]]])

In [ ]:
for summand in testWeightFactorCCDAmplitudeContractions.summandList:
    print(summand, getContractedArray(summand)[0][0,0,0,0])

1.0 * v_{p_{1}p_{0}}^{h_{1}h_{0}} 0.6452494265884844
-0.5 * v_{p_{0}p_{1}}^{h_{1}h_{0}} -0.3226247132942422
-0.5 * v_{p_{1}p_{0}}^{h_{0}h_{1}} -0.3226247132942422
1.0 * v_{p_{0}p_{1}}^{h_{0}h_{1}} 0.6452494265884844
-0.5 * f_{p_{1}}^{p_{3}}t2_{p_{0}p_{3}}^{h_{1}h_{0}} -0.03673972738562656
1.0 * f_{p_{1}}^{p_{3}}t2_{p_{3}p_{0}}^{h_{1}h_{0}} 0.07347945477125312
1.0 * f_{p_{0}}^{p_{3}}t2_{p_{1}p_{3}}^{h_{1}h_{0}} 0.07347945477125312
-0.5 * f_{p_{0}}^{p_{3}}t2_{p_{3}p_{1}}^{h_{1}h_{0}} -0.03673972738562656
1.0 * f_{p_{1}}^{p_{3}}t2_{p_{0}p_{3}}^{h_{0}h_{1}} 0.07347945477125312
-0.5 * f_{p_{1}}^{p_{3}}t2_{p_{3}p_{0}}^{h_{0}h_{1}} -0.03673972738562656
-0.5 * f_{p_{0}}^{p_{3}}t2_{p_{1}p_{3}}^{h_{0}h_{1}} -0.03673972738562656
1.0 * f_{p_{0}}^{p_{3}}t2_{p_{3}p_{1}}^{h_{0}h_{1}} 0.07347945477125312
0.5 * f_{h_{2}}^{h_{0}}t2_{p_{1}p_{0}}^{h_{2}h_{1}} 0.0819013787961804
-1.0 * f_{h_{2}}^{h_{0}}t2_{p_{0}p_{1}}^{h_{2}h_{1}} -0.1638027575923608
-1.0 * f_{h_{2}}^{h_{0}}t2_{p_{1}p_{0}}^{h_{1}h_{2}} -0.

In [ ]:
contributions = [getContractedArray(summand)[0] for summand in testWeightFactorCCDAmplitudeContractions.summandList]

In [ ]:
print(*contributions)

[[[[0.64524943]]]] [[[[-0.32262471]]]] [[[[-0.32262471]]]] [[[[0.64524943]]]] [[[[-0.03673973]]]] [[[[0.07347945]]]] [[[[0.07347945]]]] [[[[-0.03673973]]]] [[[[0.07347945]]]] [[[[-0.03673973]]]] [[[[-0.03673973]]]] [[[[0.07347945]]]] [[[[0.08190138]]]] [[[[-0.16380276]]]] [[[[-0.16380276]]]] [[[[0.08190138]]]] [[[[-0.16380276]]]] [[[[0.08190138]]]] [[[[0.08190138]]]] [[[[-0.16380276]]]] [[[[0.01166959]]]] [[[[-0.02333919]]]] [[[[-0.02333919]]]] [[[[0.01166959]]]] [[[[-0.02333919]]]] [[[[0.01166959]]]] [[[[0.01166959]]]] [[[[-0.02333919]]]] [[[[0.01125441]]]] [[[[-0.02250883]]]] [[[[-0.02250883]]]] [[[[0.01125441]]]] [[[[-0.02250883]]]] [[[[0.01125441]]]] [[[[0.01125441]]]] [[[[-0.02250883]]]] [[[[-0.01290776]]]] [[[[0.02581552]]]] [[[[0.02581552]]]] [[[[-0.05163105]]]] [[[[0.02581552]]]] [[[[-0.01290776]]]] [[[[-0.05163105]]]] [[[[0.02581552]]]] [[[[0.02581552]]]] [[[[-0.05163105]]]] [[[[-0.01290776]]]] [[[[0.02581552]]]] [[[[-0.05163105]]]] [[[[0.02581552]]]] [[[[0.02581552]]]] [[[[-0

In [ ]:
negativeContributions = [contribution for contribution in contributions if contribution[0,0,0,0] < 0]
positiveContributions = [contribution for contribution in contributions if contribution[0,0,0,0] > 0]
zeroContributions = [contribution for contribution in contributions if contribution[0,0,0,0] == 0]

In [ ]:
print(sum(negativeContributions))
print(sum(positiveContributions))

[[[[-2.67196519]]]]
[[[[2.86415612]]]]


In [92]:
print((t2 * vDiagrams[8]))
print((t2 * vDiagrams[8]).getOperator(spinFree=True))

1.0 * t2_{p_{0}p_{1}}^{h_{0}h_{1}}v_{h_{2}h_{3}}^{p_{2}p_{3}}
1.0 * a^{p_{0}\beta}a^{p_{1}\beta}a_{h_{1}\beta}a_{h_{0}\beta}a^{h_{2}\beta}a^{h_{3}\beta}a_{p_{3}\beta}a_{p_{2}\beta}
 + 1.0 * a^{p_{0}\alpha}a^{p_{1}\beta}a_{h_{1}\beta}a_{h_{0}\alpha}a^{h_{2}\beta}a^{h_{3}\beta}a_{p_{3}\beta}a_{p_{2}\beta}
 + 1.0 * a^{p_{0}\beta}a^{p_{1}\alpha}a_{h_{1}\alpha}a_{h_{0}\beta}a^{h_{2}\beta}a^{h_{3}\beta}a_{p_{3}\beta}a_{p_{2}\beta}
 + 1.0 * a^{p_{0}\alpha}a^{p_{1}\alpha}a_{h_{1}\alpha}a_{h_{0}\alpha}a^{h_{2}\beta}a^{h_{3}\beta}a_{p_{3}\beta}a_{p_{2}\beta}
 + 1.0 * a^{p_{0}\beta}a^{p_{1}\beta}a_{h_{1}\beta}a_{h_{0}\beta}a^{h_{2}\alpha}a^{h_{3}\beta}a_{p_{3}\beta}a_{p_{2}\alpha}
 + 1.0 * a^{p_{0}\alpha}a^{p_{1}\beta}a_{h_{1}\beta}a_{h_{0}\alpha}a^{h_{2}\alpha}a^{h_{3}\beta}a_{p_{3}\beta}a_{p_{2}\alpha}
 + 1.0 * a^{p_{0}\beta}a^{p_{1}\alpha}a_{h_{1}\alpha}a_{h_{0}\beta}a^{h_{2}\alpha}a^{h_{3}\beta}a_{p_{3}\beta}a_{p_{2}\alpha}
 + 1.0 * a^{p_{0}\alpha}a^{p_{1}\alpha}a_{h_{1}\alpha}a_{h_{0}\alpha}

In [93]:
print(spinFreeExcitation([Index('a', True), Index('b', True)], [Index('i', False), Index('j', False)]))

1.0 * a^{a\beta}a^{b\beta}a_{j\beta}a_{i\beta}
 + 1.0 * a^{a\alpha}a^{b\beta}a_{j\beta}a_{i\alpha}
 + 1.0 * a^{a\beta}a^{b\alpha}a_{j\alpha}a_{i\beta}
 + 1.0 * a^{a\alpha}a^{b\alpha}a_{j\alpha}a_{i\alpha}


In [ ]:
sum([getContractedArray(term)[0] for term in testWeightFactorCCDAmplitudeContractions.summandList[-32:]]) / t2.array[0,0,0,0]

array([[[[0.02220733]]]])

In [ ]:
sum([getContractedArray(term)[0] for term in testWeightFactorCCDAmplitudeContractions.summandList[-64:-32]]) / t2.array[0,0,0,0]

array([[[[0.00277592]]]])

In [ ]:
newAmplitudeEquation = evaluateWick(projectionDoubles * NormalOrderedHamiltonian + projectionDoubles * NormalOrderedHamiltonian * (t2 + 0.5 * t2 * t2), spinFree=True)

In [ ]:
disconnected = evaluateWick(0.25 * 0.25 * projectionDoubles * t2 * NormalOrderedHamiltonian * t2, spinFree=True)
for summand in disconnected.summandList:
    summand.vertexList.pop(0)
    summand.tensorList.pop(0)

In [ ]:
print(disconnected)

0.25 * t2_{p_{1}p_{0}}^{h_{1}h_{0}}v_{h_{4}h_{5}}^{p_{4}p_{5}}t2_{p_{5}p_{4}}^{h_{5}h_{4}}
 + -0.125 * t2_{p_{1}p_{0}}^{h_{1}h_{0}}v_{h_{4}h_{5}}^{p_{4}p_{5}}t2_{p_{4}p_{5}}^{h_{5}h_{4}}
 + -0.125 * t2_{p_{1}p_{0}}^{h_{1}h_{0}}v_{h_{4}h_{5}}^{p_{4}p_{5}}t2_{p_{5}p_{4}}^{h_{4}h_{5}}
 + 0.25 * t2_{p_{1}p_{0}}^{h_{1}h_{0}}v_{h_{4}h_{5}}^{p_{4}p_{5}}t2_{p_{4}p_{5}}^{h_{4}h_{5}}
 + -0.125 * t2_{p_{0}p_{1}}^{h_{1}h_{0}}v_{h_{4}h_{5}}^{p_{4}p_{5}}t2_{p_{5}p_{4}}^{h_{5}h_{4}}
 + 0.0625 * t2_{p_{0}p_{1}}^{h_{1}h_{0}}v_{h_{4}h_{5}}^{p_{4}p_{5}}t2_{p_{4}p_{5}}^{h_{5}h_{4}}
 + 0.0625 * t2_{p_{0}p_{1}}^{h_{1}h_{0}}v_{h_{4}h_{5}}^{p_{4}p_{5}}t2_{p_{5}p_{4}}^{h_{4}h_{5}}
 + -0.125 * t2_{p_{0}p_{1}}^{h_{1}h_{0}}v_{h_{4}h_{5}}^{p_{4}p_{5}}t2_{p_{4}p_{5}}^{h_{4}h_{5}}
 + -0.125 * t2_{p_{1}p_{0}}^{h_{0}h_{1}}v_{h_{4}h_{5}}^{p_{4}p_{5}}t2_{p_{5}p_{4}}^{h_{5}h_{4}}
 + 0.0625 * t2_{p_{1}p_{0}}^{h_{0}h_{1}}v_{h_{4}h_{5}}^{p_{4}p_{5}}t2_{p_{4}p_{5}}^{h_{5}h_{4}}
 + 0.0625 * t2_{p_{1}p_{0}}^{h_{0}h_{1}}v_{h_{4

In [ ]:
contractTensorSum(disconnected) / t2.array[0,0,0,0]

array([[[[-0.02220733]]]])

In [ ]:
print(newAmplitudeEquation)

1.0 * \Phi_{h_{0}h_{1}}^{p_{0}p_{1}}v_{p_{1}p_{0}}^{h_{1}h_{0}}
 + -0.5 * \Phi_{h_{0}h_{1}}^{p_{0}p_{1}}v_{p_{0}p_{1}}^{h_{1}h_{0}}
 + -0.5 * \Phi_{h_{0}h_{1}}^{p_{0}p_{1}}v_{p_{1}p_{0}}^{h_{0}h_{1}}
 + 1.0 * \Phi_{h_{0}h_{1}}^{p_{0}p_{1}}v_{p_{0}p_{1}}^{h_{0}h_{1}}
 + -2.0 * \Phi_{h_{0}h_{1}}^{p_{0}p_{1}}f_{p_{1}}^{p_{3}}t2_{p_{0}p_{3}}^{h_{1}h_{0}}
 + 4.0 * \Phi_{h_{0}h_{1}}^{p_{0}p_{1}}f_{p_{1}}^{p_{3}}t2_{p_{3}p_{0}}^{h_{1}h_{0}}
 + 4.0 * \Phi_{h_{0}h_{1}}^{p_{0}p_{1}}f_{p_{0}}^{p_{3}}t2_{p_{1}p_{3}}^{h_{1}h_{0}}
 + -2.0 * \Phi_{h_{0}h_{1}}^{p_{0}p_{1}}f_{p_{0}}^{p_{3}}t2_{p_{3}p_{1}}^{h_{1}h_{0}}
 + 4.0 * \Phi_{h_{0}h_{1}}^{p_{0}p_{1}}f_{p_{1}}^{p_{3}}t2_{p_{0}p_{3}}^{h_{0}h_{1}}
 + -2.0 * \Phi_{h_{0}h_{1}}^{p_{0}p_{1}}f_{p_{1}}^{p_{3}}t2_{p_{3}p_{0}}^{h_{0}h_{1}}
 + -2.0 * \Phi_{h_{0}h_{1}}^{p_{0}p_{1}}f_{p_{0}}^{p_{3}}t2_{p_{1}p_{3}}^{h_{0}h_{1}}
 + 4.0 * \Phi_{h_{0}h_{1}}^{p_{0}p_{1}}f_{p_{0}}^{p_{3}}t2_{p_{3}p_{1}}^{h_{0}h_{1}}
 + 2.0 * \Phi_{h_{0}h_{1}}^{p_{0}p_{1}}f_{h_{2}}^

In [ ]:
newAmplitudeTensor = copy(newAmplitudeEquation)
for summand in newAmplitudeTensor.summandList:
    summand.tensorList.pop(0)
    summand.vertexList.pop(0)

In [ ]:
len(newAmplitudeTensor.summandList)

628

In [ ]:
contractTensorSum(newAmplitudeTensor)

array([[[[-1.15475574]]]])

In [ ]:
contractTensorSum(newAmplitudeTensor) - CCDElectronicEnergy * t2.array

array([[[[-1.15781295]]]])

In [94]:
eri[0,3]

0.6452494265884848

In [ ]:
vDiagramresidualsList = []

In [ ]:
print(vDiagramresidualsList)

[<GeneralisedWick.TensorSum object at 0x11d88c220>, <GeneralisedWick.TensorSum object at 0x11d605a00>, <GeneralisedWick.TensorSum object at 0x11d88c910>, 0, 0, 0, 0, <GeneralisedWick.TensorSum object at 0x11d6686d0>, <GeneralisedWick.TensorSum object at 0x11d69b5e0>]


In [95]:
newVDiagram = Tensor('v', ['h','p'], ['h','p'])
newVDiagram.getShape(vacuum)
newVDiagram.array = h2[:Nocc, Nocc:, :Nocc, Nocc:]

In [96]:
newVDiagramsList = []
newVDiagramsList.append(Tensor('v', ['h','h'], ['h','h']))
newVDiagramsList[-1].getShape(vacuum)
newVDiagramsList[-1].array = h2[:Nocc, :Nocc, :Nocc, :Nocc]
newVDiagramsList.append(Tensor('v', ['h','h'], ['h','p']))
newVDiagramsList[-1].getShape(vacuum)
newVDiagramsList[-1].array = h2[:Nocc, :Nocc, :Nocc, Nocc:]
newVDiagramsList.append(Tensor('v', ['h','h'], ['p','h']))
newVDiagramsList[-1].getShape(vacuum)
newVDiagramsList[-1].array = h2[:Nocc, :Nocc, Nocc:, :Nocc]
newVDiagramsList.append(Tensor('v', ['h','p'], ['h','h']))
newVDiagramsList[-1].getShape(vacuum)
newVDiagramsList[-1].array = h2[:Nocc, Nocc:, :Nocc, :Nocc]
newVDiagramsList.append(Tensor('v', ['p','h'], ['h','h']))
newVDiagramsList[-1].getShape(vacuum)
newVDiagramsList[-1].array = h2[Nocc:, :Nocc, :Nocc, :Nocc]
newVDiagramsList.append(Tensor('v', ['h','h'], ['p','p']))
newVDiagramsList[-1].getShape(vacuum)
newVDiagramsList[-1].array = h2[:Nocc, :Nocc, Nocc:, Nocc:]
newVDiagramsList.append(Tensor('v', ['h','p'], ['h','p']))
newVDiagramsList[-1].getShape(vacuum)
newVDiagramsList[-1].array = h2[:Nocc, Nocc:, :Nocc, Nocc:]
newVDiagramsList.append(Tensor('v', ['p','h'], ['h','p']))
newVDiagramsList[-1].getShape(vacuum)
newVDiagramsList[-1].array = h2[Nocc:, :Nocc, :Nocc, Nocc:]
newVDiagramsList.append(Tensor('v', ['h','p'], ['p','h']))
newVDiagramsList[-1].getShape(vacuum)
newVDiagramsList[-1].array = h2[:Nocc, Nocc:, Nocc:, :Nocc]
newVDiagramsList.append(Tensor('v', ['p','h'], ['p','h']))
newVDiagramsList[-1].getShape(vacuum)
newVDiagramsList[-1].array = h2[Nocc:, :Nocc, Nocc:, :Nocc]
newVDiagramsList.append(Tensor('v', ['p','p'], ['h','h']))
newVDiagramsList[-1].getShape(vacuum)
newVDiagramsList[-1].array = h2[Nocc:, Nocc:, :Nocc, :Nocc]
newVDiagramsList.append(Tensor('v', ['h','p'], ['p','p']))
newVDiagramsList[-1].getShape(vacuum)
newVDiagramsList[-1].array = h2[:Nocc, Nocc:, Nocc:, Nocc:]
newVDiagramsList.append(Tensor('v', ['p','h'], ['p','p']))
newVDiagramsList[-1].getShape(vacuum)
newVDiagramsList[-1].array = h2[Nocc:, :Nocc, Nocc:, Nocc:]
newVDiagramsList.append(Tensor('v', ['p','p'], ['h','p']))
newVDiagramsList[-1].getShape(vacuum)
newVDiagramsList[-1].array = h2[Nocc:, Nocc:, :Nocc, Nocc:]
newVDiagramsList.append(Tensor('v', ['p','p'], ['p','h']))
newVDiagramsList[-1].getShape(vacuum)
newVDiagramsList[-1].array = h2[Nocc:, Nocc:, Nocc:, :Nocc]
newVDiagramsList.append(Tensor('v', ['p','p'], ['p','p']))
newVDiagramsList[-1].getShape(vacuum)
newVDiagramsList[-1].array = h2[Nocc:, Nocc:, Nocc:, Nocc:]

In [97]:
newVDiagramresiduals = [evaluateWick(projectionDoubles * BCHSimilarityTransform((1./2.) * d, (1./4.) * t2, 2), spinFree=True) for d in newVDiagramsList]

KeyboardInterrupt: 

In [ ]:
print(newVDiagramresiduals[6])
print(contractTensorSum(newVDiagramresiduals[6]))

-0.5 * v_{h_{2}p_{1}}^{h_{0}p_{3}}t2_{p_{0}p_{3}}^{h_{2}h_{1}}
 + 0.25 * v_{h_{2}p_{1}}^{h_{0}p_{3}}t2_{p_{3}p_{0}}^{h_{2}h_{1}}
 + 0.25 * v_{h_{2}p_{0}}^{h_{0}p_{3}}t2_{p_{1}p_{3}}^{h_{2}h_{1}}
 + -0.5 * v_{h_{2}p_{0}}^{h_{0}p_{3}}t2_{p_{3}p_{1}}^{h_{2}h_{1}}
 + 0.25 * v_{h_{2}p_{1}}^{h_{0}p_{3}}t2_{p_{0}p_{3}}^{h_{1}h_{2}}
 + -0.5 * v_{h_{2}p_{1}}^{h_{0}p_{3}}t2_{p_{3}p_{0}}^{h_{1}h_{2}}
 + -0.5 * v_{h_{2}p_{0}}^{h_{0}p_{3}}t2_{p_{1}p_{3}}^{h_{1}h_{2}}
 + 0.25 * v_{h_{2}p_{0}}^{h_{0}p_{3}}t2_{p_{3}p_{1}}^{h_{1}h_{2}}
 + 0.25 * v_{h_{2}p_{1}}^{h_{1}p_{3}}t2_{p_{0}p_{3}}^{h_{2}h_{0}}
 + -0.5 * v_{h_{2}p_{1}}^{h_{1}p_{3}}t2_{p_{3}p_{0}}^{h_{2}h_{0}}
 + -0.5 * v_{h_{2}p_{0}}^{h_{1}p_{3}}t2_{p_{1}p_{3}}^{h_{2}h_{0}}
 + 0.25 * v_{h_{2}p_{0}}^{h_{1}p_{3}}t2_{p_{3}p_{1}}^{h_{2}h_{0}}
 + -0.5 * v_{h_{2}p_{1}}^{h_{1}p_{3}}t2_{p_{0}p_{3}}^{h_{0}h_{2}}
 + 0.25 * v_{h_{2}p_{1}}^{h_{1}p_{3}}t2_{p_{3}p_{0}}^{h_{0}h_{2}}
 + 0.25 * v_{h_{2}p_{0}}^{h_{1}p_{3}}t2_{p_{1}p_{3}}^{h_{0}h_{2}}
 + -0.5 * v_{

In [ ]:
for res in newVDiagramresiduals:
    for summand in res.summandList:
        summand.tensorList.pop(0)
        summand.vertexList.pop(0)

In [ ]:
for r, res in enumerate(newVDiagramresiduals):
    if res.summandList != []:
        print(r, contractTensorSum(res))

0 [[[[-0.0900353]]]]
5 [[[[-0.00611442]]]]
6 [[[[0.05163105]]]]
7 [[[[-0.08882933]]]]
8 [[[[-0.08882933]]]]
9 [[[[0.05163105]]]]
10 [[[[1.29049885]]]]
15 [[[[-0.09335674]]]]


In [ ]:
sum([contractTensorSum(res) for res in newVDiagramresiduals if res.summandList != []])

array([[[[1.02659582]]]])

In [ ]:
vDiagramresidual = evaluateWick(projectionDoubles * BCHSimilarityTransform((1./4.) * vDiagrams[8], (1./4.) * t2, 2), spinFree=True)
#print(vDiagramresidual)

In [ ]:
newVDiagramresidual = evaluateWick(projectionDoubles * BCHSimilarityTransform((1./1.) * newVDiagram, (1./4.) * t2, 2), spinFree=True)

In [ ]:
for summand in newVDiagramresidual.summandList:
    summand.tensorList.pop(0)
    summand.vertexList.pop(0)
print(newVDiagramresidual)

-1.0 * v_{h_{2}p_{1}}^{h_{0}p_{3}}t2_{p_{0}p_{3}}^{h_{2}h_{1}}
 + 0.5 * v_{h_{2}p_{1}}^{h_{0}p_{3}}t2_{p_{3}p_{0}}^{h_{2}h_{1}}
 + 0.5 * v_{h_{2}p_{0}}^{h_{0}p_{3}}t2_{p_{1}p_{3}}^{h_{2}h_{1}}
 + -1.0 * v_{h_{2}p_{0}}^{h_{0}p_{3}}t2_{p_{3}p_{1}}^{h_{2}h_{1}}
 + 0.5 * v_{h_{2}p_{1}}^{h_{0}p_{3}}t2_{p_{0}p_{3}}^{h_{1}h_{2}}
 + -1.0 * v_{h_{2}p_{1}}^{h_{0}p_{3}}t2_{p_{3}p_{0}}^{h_{1}h_{2}}
 + -1.0 * v_{h_{2}p_{0}}^{h_{0}p_{3}}t2_{p_{1}p_{3}}^{h_{1}h_{2}}
 + 0.5 * v_{h_{2}p_{0}}^{h_{0}p_{3}}t2_{p_{3}p_{1}}^{h_{1}h_{2}}
 + 0.5 * v_{h_{2}p_{1}}^{h_{1}p_{3}}t2_{p_{0}p_{3}}^{h_{2}h_{0}}
 + -1.0 * v_{h_{2}p_{1}}^{h_{1}p_{3}}t2_{p_{3}p_{0}}^{h_{2}h_{0}}
 + -1.0 * v_{h_{2}p_{0}}^{h_{1}p_{3}}t2_{p_{1}p_{3}}^{h_{2}h_{0}}
 + 0.5 * v_{h_{2}p_{0}}^{h_{1}p_{3}}t2_{p_{3}p_{1}}^{h_{2}h_{0}}
 + -1.0 * v_{h_{2}p_{1}}^{h_{1}p_{3}}t2_{p_{0}p_{3}}^{h_{0}h_{2}}
 + 0.5 * v_{h_{2}p_{1}}^{h_{1}p_{3}}t2_{p_{3}p_{0}}^{h_{0}h_{2}}
 + 0.5 * v_{h_{2}p_{0}}^{h_{1}p_{3}}t2_{p_{1}p_{3}}^{h_{0}h_{2}}
 + -1.0 * v_{h_{2}p_{

In [ ]:
contractTensorSum(newVDiagramresidual)

array([[[[0.1032621]]]])

In [ ]:
contractTensorSum(vDiagramresidualsList[2])

array([[[[-0.05163105]]]])

In [ ]:
for summand in vDiagramresidual.summandList:
    summand.tensorList.pop(0)
    summand.vertexList.pop(0)
#print(vDiagramresidual)

In [ ]:
#vDiagramresidualsList.append(contractTensorSum(vDiagramresidual))
vDiagramresidualsList.append(vDiagramresidual)

In [ ]:
vDiagramresidualsList.append(0)

In [ ]:
len(vDiagramresidual.summandList)

608

In [ ]:
print(*vDiagramresidualsList)

[[[[-0.04667837]]]] [[[[-0.04501765]]]] [[[[-0.05163105]]]] 0 0 0 0 [[[[0.64524943]]]] [[[[-0.00305721]]]]


In [ ]:
fDiagramresidualsList = []

In [ ]:
fDiagramresidual = evaluateWick(projectionDoubles * BCHSimilarityTransform(fDiagrams[3], (1./4.) * t2, 2), spinFree=True)
print(fDiagramresidual)
for summand in fDiagramresidual.summandList:
    summand.tensorList.pop(0)
    summand.vertexList.pop(0)
print(fDiagramresidual)

0.5 * \Phi_{h_{0}h_{1}}^{p_{0}p_{1}}f_{h_{2}}^{h_{0}}t2_{p_{1}p_{0}}^{h_{2}h_{1}}
 + -1.0 * \Phi_{h_{0}h_{1}}^{p_{0}p_{1}}f_{h_{2}}^{h_{0}}t2_{p_{0}p_{1}}^{h_{2}h_{1}}
 + -1.0 * \Phi_{h_{0}h_{1}}^{p_{0}p_{1}}f_{h_{2}}^{h_{0}}t2_{p_{1}p_{0}}^{h_{1}h_{2}}
 + 0.5 * \Phi_{h_{0}h_{1}}^{p_{0}p_{1}}f_{h_{2}}^{h_{0}}t2_{p_{0}p_{1}}^{h_{1}h_{2}}
 + -1.0 * \Phi_{h_{0}h_{1}}^{p_{0}p_{1}}f_{h_{2}}^{h_{1}}t2_{p_{1}p_{0}}^{h_{2}h_{0}}
 + 0.5 * \Phi_{h_{0}h_{1}}^{p_{0}p_{1}}f_{h_{2}}^{h_{1}}t2_{p_{0}p_{1}}^{h_{2}h_{0}}
 + 0.5 * \Phi_{h_{0}h_{1}}^{p_{0}p_{1}}f_{h_{2}}^{h_{1}}t2_{p_{1}p_{0}}^{h_{0}h_{2}}
 + -1.0 * \Phi_{h_{0}h_{1}}^{p_{0}p_{1}}f_{h_{2}}^{h_{1}}t2_{p_{0}p_{1}}^{h_{0}h_{2}}
0.5 * f_{h_{2}}^{h_{0}}t2_{p_{1}p_{0}}^{h_{2}h_{1}}
 + -1.0 * f_{h_{2}}^{h_{0}}t2_{p_{0}p_{1}}^{h_{2}h_{1}}
 + -1.0 * f_{h_{2}}^{h_{0}}t2_{p_{1}p_{0}}^{h_{1}h_{2}}
 + 0.5 * f_{h_{2}}^{h_{0}}t2_{p_{0}p_{1}}^{h_{1}h_{2}}
 + -1.0 * f_{h_{2}}^{h_{1}}t2_{p_{1}p_{0}}^{h_{2}h_{0}}
 + 0.5 * f_{h_{2}}^{h_{1}}t2_{p_{0}p_{1}}^{h

In [ ]:
fDiagramresidualsList.append(fDiagramresidual)

In [ ]:
fDiagramresidualsList.append(0)

In [ ]:
print(*fDiagramresidualsList)

[[[[0.14695891]]]] 0 0 [[[[-0.32760552]]]]


In [ ]:
print(sum(vDiagramresidualsList), sum(fDiagramresidualsList))

[[[[0.49886515]]]] [[[[-0.18064661]]]]


In [ ]:
CCDElectronicEnergy * t2.array[0,0,0,0]

0.0030572092998021557

In [ ]:
def disconnectedDoubles(tensorProduct):
    for vertex in tensorProduct.vertexList:
        if set(vertex.lowerIndices) == set([('p_{0}', False), ('p_{1}', False)]) and set(vertex.upperIndices) == set([('h_{0}', True), ('h_{1}', True)]):
            return True
    return False

In [ ]:
disconnectedContributions = []
connectedContributions = []
for summand in vDiagramresidual.summandList:
    if disconnectedDoubles(summand):
        disconnectedContributions.append(getContractedArray(summand)[0])
    else:
        connectedContributions.append(getContractedArray(summand)[0])
        print(summand, getContractedArray(summand)[0], disconnectedDoubles(summand))


-0.0625 * v_{h_{2}h_{3}}^{p_{2}p_{3}}t2_{p_{1}p_{3}}^{h_{1}h_{0}}t2_{p_{0}p_{2}}^{h_{3}h_{2}} [[[[-0.0007643]]]] False
0.03125 * v_{h_{2}h_{3}}^{p_{2}p_{3}}t2_{p_{1}p_{2}}^{h_{1}h_{0}}t2_{p_{0}p_{3}}^{h_{3}h_{2}} [[[[0.00038215]]]] False
0.03125 * v_{h_{2}h_{3}}^{p_{2}p_{3}}t2_{p_{1}p_{3}}^{h_{1}h_{0}}t2_{p_{0}p_{2}}^{h_{2}h_{3}} [[[[0.00038215]]]] False
-0.0625 * v_{h_{2}h_{3}}^{p_{2}p_{3}}t2_{p_{1}p_{2}}^{h_{1}h_{0}}t2_{p_{0}p_{3}}^{h_{2}h_{3}} [[[[-0.0007643]]]] False
0.03125 * v_{h_{2}h_{3}}^{p_{2}p_{3}}t2_{p_{1}p_{3}}^{h_{1}h_{0}}t2_{p_{2}p_{0}}^{h_{3}h_{2}} [[[[0.00038215]]]] False
-0.0625 * v_{h_{2}h_{3}}^{p_{2}p_{3}}t2_{p_{1}p_{2}}^{h_{1}h_{0}}t2_{p_{3}p_{0}}^{h_{3}h_{2}} [[[[-0.0007643]]]] False
-0.0625 * v_{h_{2}h_{3}}^{p_{2}p_{3}}t2_{p_{1}p_{3}}^{h_{1}h_{0}}t2_{p_{2}p_{0}}^{h_{2}h_{3}} [[[[-0.0007643]]]] False
0.03125 * v_{h_{2}h_{3}}^{p_{2}p_{3}}t2_{p_{1}p_{2}}^{h_{1}h_{0}}t2_{p_{3}p_{0}}^{h_{2}h_{3}} [[[[0.00038215]]]] False
0.03125 * v_{h_{2}h_{3}}^{p_{2}p_{3}}t2_{p_{3}p_

In [ ]:
print(connectedContributions)

[array([[[[-0.0007643]]]]), array([[[[0.00038215]]]]), array([[[[0.00038215]]]]), array([[[[-0.0007643]]]]), array([[[[0.00038215]]]]), array([[[[-0.0007643]]]]), array([[[[-0.0007643]]]]), array([[[[0.00038215]]]]), array([[[[0.00038215]]]]), array([[[[-0.00019108]]]]), array([[[[-0.00019108]]]]), array([[[[0.00038215]]]]), array([[[[-0.00019108]]]]), array([[[[0.00038215]]]]), array([[[[0.00038215]]]]), array([[[[-0.00019108]]]]), array([[[[0.00038215]]]]), array([[[[-0.00019108]]]]), array([[[[-0.00019108]]]]), array([[[[0.00038215]]]]), array([[[[-0.0007643]]]]), array([[[[0.00038215]]]]), array([[[[0.00038215]]]]), array([[[[-0.0007643]]]]), array([[[[0.00038215]]]]), array([[[[-0.00019108]]]]), array([[[[-0.00019108]]]]), array([[[[0.00038215]]]]), array([[[[-0.00019108]]]]), array([[[[0.00038215]]]]), array([[[[0.00038215]]]]), array([[[[-0.00019108]]]]), array([[[[0.00038215]]]]), array([[[[-0.0007643]]]]), array([[[[-0.0007643]]]]), array([[[[0.00038215]]]]), array([[[[-0.0001

In [ ]:
vDiagrams[7].array

array([[[[0.64524943]]]])

In [ ]:
contractTensorSum(vDiagramresidualsList[0]) + contractTensorSum(vDiagramresidualsList[1]) + contractTensorSum(vDiagramresidualsList[2]) + contractTensorSum(vDiagramresidualsList[7]) + contractTensorSum(vDiagramresidualsList[8])

array([[[[0.37283753]]]])

In [ ]:
contractTensorSum(fDiagramresidualsList[0]) + contractTensorSum(fDiagramresidualsList[3])

array([[[[-0.18064661]]]])

In [98]:
testSlicing = Tensor('test', ['g','g'], ['g','g'])

In [99]:
print(testSlicing.getDiagrams(vacuum))

['h', 'h']
['h', 'h']
['h', 'h']
['h', 'h']
[slice(None, 1, None), slice(None, 1, None), slice(None, 1, None), slice(None, 1, None)]


AttributeError: 'Tensor' object has no attribute 'array'

In [100]:
t = trueAmplitude

In [101]:
C = h2[1,1,0,0]
L1 = t * h1[1,1]
L2 = t * h1[0,0]
L3 = t * h2[1,1,1,1]
L4 = t * h2[0,0,0,0]
L5 = t * h2[0,1,1,0]
L6 = t * h2[0,1,0,1]
Q = t * t * h2[0,0,1,1]
print(C, L1, L2, L3, L4, L5, L6, Q)

0.6452494265884844 0.07347945477125312 0.1638027575923608 -0.09335674174028416 -0.09003530340403822 -0.025815524319352173 -0.025815524319352173 0.012228837199208632


In [102]:
t * h2[1,1,1,1] + t * h2[0,0,0,0]

-0.1833920451443224

In [103]:
(2 * C + 2 * L1 - 2 * L2 + 2 * L3 + 2 * L4 + 4 * L5 - 4 * L6)/Q

60.76359879042259

In [104]:
C * (4-2) + L1 * (4-2) + L2 * (-4+2) + L3 * (0.5*4-0.5*2) + L4 * (0.5*4-0.5*2) + L5 * (8-4-4+8) + L6 * (-4+2+2-4) + Q * (0.25*4-0.25*2+16-8-8+4-0.5*8+0.5*4+0.5*4-0.5*2-0.5*8+0.5*4+0.5*4-0.5*2)

0.8537701981110437

In [105]:
C + L1 + L2 + L3 + L4 + L5 * (2-2+2-2) + L6 * (-4+2+2-4) + Q * (0.25*4-0.25*2+16-8-8+4-0.5*8+0.5*4+0.5*4-0.5*2-0.5*8+0.5*4+0.5*4-0.5*2)

0.8329737840832061

In [106]:
h2

array([[[[6.54009511e-01, 1.11022302e-16],
         [8.32667268e-17, 6.45249427e-01]],

        [[1.17961196e-16, 1.87521981e-01],
         [1.87521981e-01, 1.11022302e-16]]],


       [[[1.04083409e-16, 1.87521981e-01],
         [1.87521981e-01, 5.55111512e-17]],

        [[6.45249427e-01, 2.22044605e-16],
         [1.38777878e-16, 6.78136184e-01]]]])

In [107]:
h2Tensor = Tensor('v', ['g','g'], ['g','g'])
h2Tensor.getShape(vacuum)
h2Tensor.array = h2

In [108]:
def getDiagrams(tensor, vacuum):
    Nocc = sum(vacuum)
    Norbs = len(vacuum)
    diagrams = []
    lowerGeneralIndexCount = sum(i == 'g' for i in tensor.lowerIndexTypes)
    lowerSplits = list(itertools.product(['h', 'p'], repeat=lowerGeneralIndexCount))
    upperGeneralIndexCount = sum(i == 'g' for i in tensor.upperIndexTypes)
    upperSplits = list(itertools.product(['h', 'p'], repeat=upperGeneralIndexCount))
    for lowerSplit in lowerSplits:
        lowerSlices = [slice(None)] * tensor.excitationRank
        lowerSplitIndexTypes = list(lowerSplit)
        lGI = 0
        newLowerIndexTypes = copy(tensor.lowerIndexTypes)
        for lI in range(len(newLowerIndexTypes)):
            if newLowerIndexTypes[lI] == 'g':
                newLI = lowerSplitIndexTypes[lGI]
                if newLI == 'h':
                    lowerSlices[lI] = slice(None,Nocc)
                elif newLI == 'p':
                    lowerSlices[lI] = slice(Nocc, None)
                newLowerIndexTypes[lI] = newLI
                lGI += 1
        for upperSplit in upperSplits:
            print(lowerSplit, upperSplit)
            upperSlices = [slice(None)] * tensor.excitationRank
            upperSplitIndexTypes = list(upperSplit)
            uGI = 0
            newUpperIndexTypes = copy(tensor.upperIndexTypes)
            for uI in range(len(newUpperIndexTypes)):
                if newUpperIndexTypes[uI] == 'g':
                    newUI = upperSplitIndexTypes[uGI]
                    if newUI == 'h':
                        upperSlices[uI] = slice(None,Nocc)
                    elif newUI == 'p':
                        upperSlices[uI] = slice(Nocc, None)
                    newUpperIndexTypes[uI] = newUI
                    uGI += 1
            slices = tuple(lowerSlices + upperSlices)
            print(lowerSplitIndexTypes)
            print(upperSplitIndexTypes)
            print(newLowerIndexTypes)
            print(newUpperIndexTypes)
            print(slices)
            diagram = Tensor(tensor.name, newLowerIndexTypes, newUpperIndexTypes)
            diagram.array = tensor.array[slices]
            diagrams.append(diagram)
    return diagrams

In [109]:
h2Terms = getDiagrams(h2Tensor, vacuum)

('h', 'h') ('h', 'h')
['h', 'h']
['h', 'h']
['h', 'h']
['h', 'h']
(slice(None, 1, None), slice(None, 1, None), slice(None, 1, None), slice(None, 1, None))
('h', 'h') ('h', 'p')
['h', 'h']
['h', 'p']
['h', 'h']
['h', 'p']
(slice(None, 1, None), slice(None, 1, None), slice(None, 1, None), slice(1, None, None))
('h', 'h') ('p', 'h')
['h', 'h']
['p', 'h']
['h', 'h']
['p', 'h']
(slice(None, 1, None), slice(None, 1, None), slice(1, None, None), slice(None, 1, None))
('h', 'h') ('p', 'p')
['h', 'h']
['p', 'p']
['h', 'h']
['p', 'p']
(slice(None, 1, None), slice(None, 1, None), slice(1, None, None), slice(1, None, None))
('h', 'p') ('h', 'h')
['h', 'p']
['h', 'h']
['h', 'p']
['h', 'h']
(slice(None, 1, None), slice(1, None, None), slice(None, 1, None), slice(None, 1, None))
('h', 'p') ('h', 'p')
['h', 'p']
['h', 'p']
['h', 'p']
['h', 'p']
(slice(None, 1, None), slice(1, None, None), slice(None, 1, None), slice(1, None, None))
('h', 'p') ('p', 'h')
['h', 'p']
['p', 'h']
['h', 'p']
['p', 'h']
(sli

In [110]:
h1Tensor = Tensor('f', ['g'], ['g'])
h1Tensor.getShape(vacuum)
h1Tensor.array = h1

In [111]:
h1Terms = getDiagrams(h1Tensor, vacuum)

('h',) ('h',)
['h']
['h']
['h']
['h']
(slice(None, 1, None), slice(None, 1, None))
('h',) ('p',)
['h']
['p']
['h']
['p']
(slice(None, 1, None), slice(1, None, None))
('p',) ('h',)
['p']
['h']
['p']
['h']
(slice(1, None, None), slice(None, 1, None))
('p',) ('p',)
['p']
['p']
['p']
['p']
(slice(1, None, None), slice(1, None, None))


In [112]:
Hamiltonian = sum(h1Terms) + 0.5 * sum(h2Terms)

In [113]:
print(Hamiltonian)

1.0 * f_{h_{0}}^{h_{1}}
 + 1.0 * f_{h_{0}}^{p_{0}}
 + 1.0 * f_{p_{0}}^{h_{0}}
 + 1.0 * f_{p_{0}}^{p_{1}}
 + 0.5 * v_{h_{0}h_{1}}^{h_{2}h_{3}}
 + 0.5 * v_{h_{0}h_{1}}^{h_{2}p_{0}}
 + 0.5 * v_{h_{0}h_{1}}^{p_{0}h_{2}}
 + 0.5 * v_{h_{0}h_{1}}^{p_{0}p_{1}}
 + 0.5 * v_{h_{0}p_{0}}^{h_{1}h_{2}}
 + 0.5 * v_{h_{0}p_{0}}^{h_{1}p_{1}}
 + 0.5 * v_{h_{0}p_{0}}^{p_{1}h_{1}}
 + 0.5 * v_{h_{0}p_{0}}^{p_{1}p_{2}}
 + 0.5 * v_{p_{0}h_{0}}^{h_{1}h_{2}}
 + 0.5 * v_{p_{0}h_{0}}^{h_{1}p_{1}}
 + 0.5 * v_{p_{0}h_{0}}^{p_{1}h_{1}}
 + 0.5 * v_{p_{0}h_{0}}^{p_{1}p_{2}}
 + 0.5 * v_{p_{0}p_{1}}^{h_{0}h_{1}}
 + 0.5 * v_{p_{0}p_{1}}^{h_{0}p_{2}}
 + 0.5 * v_{p_{0}p_{1}}^{p_{2}h_{0}}
 + 0.5 * v_{p_{0}p_{1}}^{p_{2}p_{3}}


In [114]:
fullySplitCCDEnergyEquation = evaluateWick(BCHSimilarityTransform(Hamiltonian, (1./4.) * t2, 2), spinFree=True)

In [115]:
print(BCHSimilarityTransform(Hamiltonian, (1./4.) * t2, 2))

1.0 * f_{h_{0}}^{h_{1}}
 + 1.0 * f_{h_{0}}^{p_{0}}
 + 1.0 * f_{p_{0}}^{h_{0}}
 + 1.0 * f_{p_{0}}^{p_{1}}
 + 0.5 * v_{h_{0}h_{1}}^{h_{2}h_{3}}
 + 0.5 * v_{h_{0}h_{1}}^{h_{2}p_{0}}
 + 0.5 * v_{h_{0}h_{1}}^{p_{0}h_{2}}
 + 0.5 * v_{h_{0}h_{1}}^{p_{0}p_{1}}
 + 0.5 * v_{h_{0}p_{0}}^{h_{1}h_{2}}
 + 0.5 * v_{h_{0}p_{0}}^{h_{1}p_{1}}
 + 0.5 * v_{h_{0}p_{0}}^{p_{1}h_{1}}
 + 0.5 * v_{h_{0}p_{0}}^{p_{1}p_{2}}
 + 0.5 * v_{p_{0}h_{0}}^{h_{1}h_{2}}
 + 0.5 * v_{p_{0}h_{0}}^{h_{1}p_{1}}
 + 0.5 * v_{p_{0}h_{0}}^{p_{1}h_{1}}
 + 0.5 * v_{p_{0}h_{0}}^{p_{1}p_{2}}
 + 0.5 * v_{p_{0}p_{1}}^{h_{0}h_{1}}
 + 0.5 * v_{p_{0}p_{1}}^{h_{0}p_{2}}
 + 0.5 * v_{p_{0}p_{1}}^{p_{2}h_{0}}
 + 0.5 * v_{p_{0}p_{1}}^{p_{2}p_{3}}
 + 0.25 * f_{h_{0}}^{h_{1}}t2_{p_{0}p_{1}}^{h_{2}h_{3}}
 + 0.25 * f_{h_{0}}^{p_{0}}t2_{p_{1}p_{2}}^{h_{1}h_{2}}
 + 0.25 * f_{p_{0}}^{h_{0}}t2_{p_{1}p_{2}}^{h_{1}h_{2}}
 + 0.25 * f_{p_{0}}^{p_{1}}t2_{p_{2}p_{3}}^{h_{0}h_{1}}
 + 0.125 * v_{h_{0}h_{1}}^{h_{2}h_{3}}t2_{p_{0}p_{1}}^{h_{4}h_{5}}
 + 0.125 * v

In [116]:
print(fullySplitCCDEnergyEquation)

0.5 * v_{h_{0}h_{1}}^{p_{0}p_{1}}t2_{p_{1}p_{0}}^{h_{1}h_{0}}
 + -0.25 * v_{h_{0}h_{1}}^{p_{0}p_{1}}t2_{p_{0}p_{1}}^{h_{1}h_{0}}
 + -0.25 * v_{h_{0}h_{1}}^{p_{0}p_{1}}t2_{p_{1}p_{0}}^{h_{0}h_{1}}
 + 0.5 * v_{h_{0}h_{1}}^{p_{0}p_{1}}t2_{p_{0}p_{1}}^{h_{0}h_{1}}


In [117]:
t2.array[0,0,0,0] = trueAmplitude

In [118]:
fullySplitCCDProjectedAmplitudes = evaluateWick(projectionDoubles * BCHSimilarityTransform(Hamiltonian, (1./4.) * t2, 2), spinFree=True)

In [119]:
print(fullySplitCCDProjectedAmplitudes)

2.0 * \Phi_{h_{0}h_{1}}^{p_{0}p_{1}}v_{p_{1}p_{0}}^{h_{1}h_{0}}
 + -1.0 * \Phi_{h_{0}h_{1}}^{p_{0}p_{1}}v_{p_{0}p_{1}}^{h_{1}h_{0}}
 + -1.0 * \Phi_{h_{0}h_{1}}^{p_{0}p_{1}}v_{p_{1}p_{0}}^{h_{0}h_{1}}
 + 2.0 * \Phi_{h_{0}h_{1}}^{p_{0}p_{1}}v_{p_{0}p_{1}}^{h_{0}h_{1}}
 + 0.5 * \Phi_{h_{0}h_{1}}^{p_{0}p_{1}}f_{h_{2}}^{h_{0}}t2_{p_{1}p_{0}}^{h_{2}h_{1}}
 + -1.0 * \Phi_{h_{0}h_{1}}^{p_{0}p_{1}}f_{h_{2}}^{h_{0}}t2_{p_{0}p_{1}}^{h_{2}h_{1}}
 + -1.0 * \Phi_{h_{0}h_{1}}^{p_{0}p_{1}}f_{h_{2}}^{h_{0}}t2_{p_{1}p_{0}}^{h_{1}h_{2}}
 + 0.5 * \Phi_{h_{0}h_{1}}^{p_{0}p_{1}}f_{h_{2}}^{h_{0}}t2_{p_{0}p_{1}}^{h_{1}h_{2}}
 + -1.0 * \Phi_{h_{0}h_{1}}^{p_{0}p_{1}}f_{h_{2}}^{h_{1}}t2_{p_{1}p_{0}}^{h_{2}h_{0}}
 + 0.5 * \Phi_{h_{0}h_{1}}^{p_{0}p_{1}}f_{h_{2}}^{h_{1}}t2_{p_{0}p_{1}}^{h_{2}h_{0}}
 + 0.5 * \Phi_{h_{0}h_{1}}^{p_{0}p_{1}}f_{h_{2}}^{h_{1}}t2_{p_{1}p_{0}}^{h_{0}h_{2}}
 + -1.0 * \Phi_{h_{0}h_{1}}^{p_{0}p_{1}}f_{h_{2}}^{h_{1}}t2_{p_{0}p_{1}}^{h_{0}h_{2}}
 + -0.5 * \Phi_{h_{0}h_{1}}^{p_{0}p_{1}}f_{p_{1}}

In [120]:
for summand in fullySplitCCDProjectedAmplitudes.summandList:
    summand.vertexList.pop(0)
    summand.tensorList.pop(0)

In [199]:
contractTensorSum(fullySplitCCDEnergyEquation)

-0.044414665907314056

In [200]:
contractTensorSum(fullySplitCCDProjectedAmplitudes)

array([[[[0.86871474]]]])

In [123]:
len(fullySplitCCDProjectedAmplitudes.summandList)

708

In [124]:
print(evaluateWick(Hamiltonian, spinFree=True))

In [142]:
constantAmplitudeTerms = TensorSum([summand for summand in fullySplitCCDProjectedAmplitudes.summandList if len(summand.tensorList) == 1])
linearAmplitudeTerms = TensorSum([summand for summand in fullySplitCCDProjectedAmplitudes.summandList if len(summand.tensorList) == 2])
quadraticAmplitudeTerms = TensorSum([summand for summand in fullySplitCCDProjectedAmplitudes.summandList if len(summand.tensorList) == 3])

In [197]:
contractTensorSum(quadraticAmplitudeTerms) * (1. / trueAmplitude)

array([[[[0.04441467]]]])

In [198]:
h1

array([[-1.18985062e+00,  2.60021255e-17],
       [-4.71423569e-17, -5.33749102e-01]])

In [259]:
h2Tensor.array = get2bodyHamiltonianArray(mf)
#h2Tensor.array = h2.swapaxes(2,3).swapaxes(1,2)

In [266]:
eri.reshape(2,2,2,2)

array([[[[6.54009511e-01, 1.11022302e-16],
         [8.32667268e-17, 6.45249427e-01]],

        [[1.17961196e-16, 0.00000000e+00],
         [1.87521981e-01, 1.11022302e-16]]],


       [[[1.04083409e-16, 1.87521981e-01],
         [0.00000000e+00, 5.55111512e-17]],

        [[6.45249427e-01, 2.22044605e-16],
         [1.38777878e-16, 6.78136184e-01]]]])

In [271]:
h2Tensor.array = ao2mo.kernel(mol, mf.mo_coeff, compact=False).reshape(2,2,2,2).swapaxes(2,3).swapaxes(1,2)

In [184]:
h2Tensor.array = test

In [272]:
newh2Terms = getDiagrams(h2Tensor, vacuum)

('h', 'h') ('h', 'h')
['h', 'h']
['h', 'h']
['h', 'h']
['h', 'h']
(slice(None, 1, None), slice(None, 1, None), slice(None, 1, None), slice(None, 1, None))
('h', 'h') ('h', 'p')
['h', 'h']
['h', 'p']
['h', 'h']
['h', 'p']
(slice(None, 1, None), slice(None, 1, None), slice(None, 1, None), slice(1, None, None))
('h', 'h') ('p', 'h')
['h', 'h']
['p', 'h']
['h', 'h']
['p', 'h']
(slice(None, 1, None), slice(None, 1, None), slice(1, None, None), slice(None, 1, None))
('h', 'h') ('p', 'p')
['h', 'h']
['p', 'p']
['h', 'h']
['p', 'p']
(slice(None, 1, None), slice(None, 1, None), slice(1, None, None), slice(1, None, None))
('h', 'p') ('h', 'h')
['h', 'p']
['h', 'h']
['h', 'p']
['h', 'h']
(slice(None, 1, None), slice(1, None, None), slice(None, 1, None), slice(None, 1, None))
('h', 'p') ('h', 'p')
['h', 'p']
['h', 'p']
['h', 'p']
['h', 'p']
(slice(None, 1, None), slice(1, None, None), slice(None, 1, None), slice(1, None, None))
('h', 'p') ('p', 'h')
['h', 'p']
['p', 'h']
['h', 'p']
['p', 'h']
(sli

In [273]:
for t, term in enumerate(h2Terms):
    term.array = newh2Terms[t].array

In [241]:
print(fullySplitCCDEnergyEquation)

0.5 * v_{h_{0}h_{1}}^{p_{0}p_{1}}t2_{p_{1}p_{0}}^{h_{1}h_{0}}
 + -0.25 * v_{h_{0}h_{1}}^{p_{0}p_{1}}t2_{p_{0}p_{1}}^{h_{1}h_{0}}
 + -0.25 * v_{h_{0}h_{1}}^{p_{0}p_{1}}t2_{p_{1}p_{0}}^{h_{0}h_{1}}
 + 0.5 * v_{h_{0}h_{1}}^{p_{0}p_{1}}t2_{p_{0}p_{1}}^{h_{0}h_{1}}


In [138]:
fullySplitCCSDEnergyEquation = evaluateWick(BCHSimilarityTransform(Hamiltonian, t1 + (1./4.) * t2, 2), spinFree=True)

In [130]:
t1.name = "{t_{1}}"
t2.name = "{t_{2}}"

In [139]:
print(fullySplitCCSDEnergyEquation)

2.0 * f_{h_{0}}^{p_{0}}{t_{1}}_{p_{0}}^{h_{0}}
 + 0.5 * v_{h_{0}h_{1}}^{p_{0}p_{1}}{t_{2}}_{p_{1}p_{0}}^{h_{1}h_{0}}
 + -0.25 * v_{h_{0}h_{1}}^{p_{0}p_{1}}{t_{2}}_{p_{0}p_{1}}^{h_{1}h_{0}}
 + -0.25 * v_{h_{0}h_{1}}^{p_{0}p_{1}}{t_{2}}_{p_{1}p_{0}}^{h_{0}h_{1}}
 + 0.5 * v_{h_{0}h_{1}}^{p_{0}p_{1}}{t_{2}}_{p_{0}p_{1}}^{h_{0}h_{1}}
 + -0.5 * v_{h_{0}h_{1}}^{p_{0}p_{1}}{t_{1}}_{p_{1}}^{h_{0}}{t_{1}}_{p_{0}}^{h_{1}}
 + 1.0 * v_{h_{0}h_{1}}^{p_{0}p_{1}}{t_{1}}_{p_{0}}^{h_{0}}{t_{1}}_{p_{1}}^{h_{1}}
 + 1.0 * v_{h_{0}h_{1}}^{p_{0}p_{1}}{t_{1}}_{p_{1}}^{h_{1}}{t_{1}}_{p_{0}}^{h_{0}}
 + -0.5 * v_{h_{0}h_{1}}^{p_{0}p_{1}}{t_{1}}_{p_{0}}^{h_{1}}{t_{1}}_{p_{1}}^{h_{0}}


In [157]:
(h2 * 0.5).swapaxes(1,2)[1,1,0,0]

0.09376099045135429

In [132]:
print(Hamiltonian.getOperator(spinFree=True))

-1.0 * a_{h_{1}\beta}a^{h_{0}\beta}
 + -1.0 * a_{h_{1}\alpha}a^{h_{0}\alpha}
 + 1.0 * a^{h_{0}\beta}a_{p_{0}\beta}
 + 1.0 * a^{h_{0}\alpha}a_{p_{0}\alpha}
 + 1.0 * a^{p_{0}\beta}a_{h_{0}\beta}
 + 1.0 * a^{p_{0}\alpha}a_{h_{0}\alpha}
 + 1.0 * a^{p_{0}\beta}a_{p_{1}\beta}
 + 1.0 * a^{p_{0}\alpha}a_{p_{1}\alpha}
 + 1.0 * a_{h_{3}\beta}a_{h_{2}\beta}a^{h_{0}\beta}a^{h_{1}\beta}
 + 1.0 * a_{h_{3}\beta}a_{h_{2}\alpha}a^{h_{0}\alpha}a^{h_{1}\beta}
 + 1.0 * a_{h_{3}\alpha}a_{h_{2}\beta}a^{h_{0}\beta}a^{h_{1}\alpha}
 + 1.0 * a_{h_{3}\alpha}a_{h_{2}\alpha}a^{h_{0}\alpha}a^{h_{1}\alpha}
 + 0.0 * a_{h_{2}\beta}a^{h_{0}\beta}a^{h_{1}\beta}a_{p_{0}\beta}
 + -1.0 * a_{h_{2}\alpha}a^{h_{0}\alpha}a^{h_{1}\beta}a_{p_{0}\beta}
 + -1.0 * a_{h_{2}\beta}a^{h_{0}\beta}a^{h_{1}\alpha}a_{p_{0}\alpha}
 + 0.0 * a_{h_{2}\alpha}a^{h_{0}\alpha}a^{h_{1}\alpha}a_{p_{0}\alpha}
 + 1.0 * a_{h_{2}\beta}a^{h_{0}\alpha}a^{h_{1}\beta}a_{p_{0}\alpha}
 + 1.0 * a_{h_{2}\alpha}a^{h_{0}\beta}a^{h_{1}\alpha}a_{p_{0}\beta}
 + 1.0 

In [133]:
print(Hamiltonian)

1.0 * f_{h_{0}}^{h_{1}}
 + 1.0 * f_{h_{0}}^{p_{0}}
 + 1.0 * f_{p_{0}}^{h_{0}}
 + 1.0 * f_{p_{0}}^{p_{1}}
 + 0.5 * v_{h_{0}h_{1}}^{h_{2}h_{3}}
 + 0.5 * v_{h_{0}h_{1}}^{h_{2}p_{0}}
 + 0.5 * v_{h_{0}h_{1}}^{p_{0}h_{2}}
 + 0.5 * v_{h_{0}h_{1}}^{p_{0}p_{1}}
 + 0.5 * v_{h_{0}p_{0}}^{h_{1}h_{2}}
 + 0.5 * v_{h_{0}p_{0}}^{h_{1}p_{1}}
 + 0.5 * v_{h_{0}p_{0}}^{p_{1}h_{1}}
 + 0.5 * v_{h_{0}p_{0}}^{p_{1}p_{2}}
 + 0.5 * v_{p_{0}h_{0}}^{h_{1}h_{2}}
 + 0.5 * v_{p_{0}h_{0}}^{h_{1}p_{1}}
 + 0.5 * v_{p_{0}h_{0}}^{p_{1}h_{1}}
 + 0.5 * v_{p_{0}h_{0}}^{p_{1}p_{2}}
 + 0.5 * v_{p_{0}p_{1}}^{h_{0}h_{1}}
 + 0.5 * v_{p_{0}p_{1}}^{h_{0}p_{2}}
 + 0.5 * v_{p_{0}p_{1}}^{p_{2}h_{0}}
 + 0.5 * v_{p_{0}p_{1}}^{p_{2}p_{3}}


In [134]:
h1

array([[-1.18985062e+00,  2.60021255e-17],
       [-4.71423569e-17, -5.33749102e-01]])

In [135]:
projectionDoubles.array = np.ones_like(t2.array)

In [275]:
t2.array[0,0,0,0]= trueAmplitude

In [277]:
contractTensorSum(fullySplitCCDEnergyEquation)

-0.012907762159676086

In [182]:
test = h2.swapaxes(2,3).swapaxes(1,2)

In [183]:
test[0,1,1,0]=0
test[1,0,0,1]=0

-0.13766665750371915

In [258]:
def get2bodyHamiltonianArray(mf_):
    eri = ao2mo.kernel(mf_.mol, mf_.mo_coeff)
    Norbs_ = mf_.mol.nao
    hamiltonian2BodyArray = np.zeros((Norbs_,Norbs_,Norbs_,Norbs_))
    for p in range(Norbs_):
        for q in range(Norbs_):
            for r in range(p + 1):
                for s in range(q + 1):
                    x = int(p + Norbs_ * r - 0.5 * r * (r + 1))
                    y = int(q + Norbs_ * s - 0.5 * s * (s + 1))
                    if p == r and q == s:
                        hamiltonian2BodyArray[p,q,r,s] += eri[x, y]
                    else:
                        hamiltonian2BodyArray[p,q,r,s] += eri[x, y]
                        hamiltonian2BodyArray[r,s,p,q] += np.conjugate(eri[x, y])
    return hamiltonian2BodyArray

In [246]:
cisolver.kernel()

(-1.1284543355083052,
 array([[ 9.90656547e-01,  2.77555756e-17],
        [ 1.91460513e-17, -1.36380375e-01]]))

In [249]:
(cisolver.e_tot - mf.e_tot) / trueAmplitude

0.18752198088258215

In [264]:
h2Tensor.array

array([[[[6.54009511e-01, 8.32667268e-17],
         [1.04083409e-16, 1.87521981e-01]],

        [[8.32667268e-17, 6.45249427e-01],
         [0.00000000e+00, 5.55111512e-17]]],


       [[[1.04083409e-16, 0.00000000e+00],
         [6.45249427e-01, 1.38777878e-16]],

        [[1.87521981e-01, 5.55111512e-17],
         [1.38777878e-16, 6.78136184e-01]]]])